In [1]:
import pandas as pd
import numpy as np
import matplotlib
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

pd.set_option("display.precision", 8)

In [2]:
pickle_dir = '../input/pickle'
metadata_dir = '../input/metadata'
csv_dir = '../input/csv'

In [9]:
material_runrate = pd.read_pickle(os.path.join(pickle_dir, 'production_capacity'))
material_runrate = material_runrate[['Plant', 'Line','Material', 'BaseUnitsPerHour']].round(2)
material_runrate

,Plant,Line,Material,BaseUnitsPerHour
0,5501,LN01,216789,15748.03
1,5501,LN01,277956,10626.99
2,5501,LN01,407611,10626.99
3,5501,LN01,407733,10626.99
4,5501,LN01,465044,4960.32
...,...,...,...,...
9365,5567,LN01,267378,10000.00
9366,5567,LN01,271170,10000.00
9367,5567,LN01,272764,10000.00
9368,5567,LN01,277002,10000.00


In [7]:
material_runrate.loc[(material_runrate.Summary_Portfolio == 'Cans - Singles')
                    &(material_runrate.Plant == 5501)
                    &(material_runrate.Line == 'LN01')]

AttributeError: 'DataFrame' object has no attribute 'Summary_Portfolio'

In [54]:
# Runrate file

material_snp_mapping = pd.read_csv(os.path.join(metadata_dir, 'material_snp_mapping.csv'))
runrate = pd.read_csv(os.path.join(metadata_dir, 'runrate.csv'))
snp_portfolio_mapping = pd.read_csv(os.path.join(metadata_dir, 'snp_portfolio_mapping.csv'))


material_runrate = pd.merge(pd.merge(material_snp_mapping, snp_portfolio_mapping, on='SNPPL01', how='inner'),
                            runrate[['Material', 'BaseUnitsPerHour', 'Plant', 'Line']], on='Material', how='inner')

portfolio_runrate = material_runrate[['Plant', 'Line','Summary_Portfolio', 'BaseUnitsPerHour']]\
                    .groupby(['Plant', 'Line','Summary_Portfolio']).mean().round(2).reset_index()

In [55]:
portfolio_runrate

,Plant,Line,Summary_Portfolio,BaseUnitsPerHour
0,5501,LN01,Cans - Shrink,6433.33
1,5501,LN01,Cans - Singles,5013.44
2,5501,LN01,Monster,4961.53
3,5501,LN01,Relentless,5266.64
4,5501,LN02,REPACK,2177.74
...,...,...,...,...
196,5559,LN01,REPACK,9012.83
197,5560,LN01,Cans - Board,10000.00
198,5560,LN01,REPACK,7919.78
199,5567,LN01,Cans - Board,10000.00


In [3]:
# Check where supply short fall

dP_siteSummarised = pd.read_csv(os.path.join(csv_dir, 'dP_siteSummarised.csv')).round(2)

#Manuall add impurity to Aseptic  
conditions_demand_zuc = ((dP_siteSummarised.Site_SAP == 5502) \
            &(dP_siteSummarised.Level2Region == 'North') \
            &(dP_siteSummarised.Year_Period == '2020_06') \
            &(dP_siteSummarised.KeyField.isin(['Available'])) \
            &(dP_siteSummarised.Uom.isin(['ZUC'])) \
            &(dP_siteSummarised.Summary_Portfolio == 'Aseptic')
             )

conditions_demand_zrw = ((dP_siteSummarised.Site_SAP == 5502) \
            &(dP_siteSummarised.Level2Region == 'North') \
            &(dP_siteSummarised.Year_Period == '2020_06') \
            &(dP_siteSummarised.KeyField.isin(['Available'])) \
            &(dP_siteSummarised.Uom.isin(['ZRW'])) \
            &(dP_siteSummarised.Summary_Portfolio == 'Aseptic')
             )

zuc_zrw_ratio = 84895.00 / 79015.33
new_zuc = float(dP_siteSummarised.loc[conditions_demand_zuc, 'Value']) - 10000
new_zrw = 84895.00 / zuc_zrw_ratio

print(new_zuc)
print(new_zrw)

dP_siteSummarised.loc[conditions_demand_zuc, 'Value'] = new_zuc
dP_siteSummarised.loc[conditions_demand_zrw, 'Value'] = new_zrw


dP_siteSummarised.to_pickle(os.path.join(pickle_dir, 'dP_siteSummarised'))

74892.43
79015.33


In [108]:
['Summary_Portfolio'] +list(dP_siteSummarised.Uom.unique())

['Summary_Portfolio', 'ZRW', 'ZUC']

In [98]:
mP_Summarised.Uom.unique()

array(['GBP', 'PAL', 'Trucks', 'ZRW', 'ZUC'], dtype=object)

In [114]:
x = dP_siteSummarised[['Summary_Portfolio', 'Uom', 'Value']].drop_duplicates(subset=['Summary_Portfolio', 'Uom']).pivot(
                                    index=['Summary_Portfolio', ],
                                    columns='Uom')['Value'].reset_index()

x['ratio'] = x['ZUC'] / x['ZRW']

x

Uom,Summary_Portfolio,ZRW,ZUC,ratio
0,Aseptic,79015.33,74895.0,0.94785404
1,Cans - Board,2654606.00,4270769.6,1.60881487
2,Cans - Shrink,1218885.35,1700136.0,1.39482848
3,Cans - Singles,1313877.67,1832634.0,1.39482849
4,Capri Sun,339014.77,347706.0,1.02563673
5,GB,47911.75,100243.0,2.09224251
6,Glass,421262.69,414358.0,0.98360954
7,LPET,3382895.41,6520608.0,1.92752279
8,Monster,1025672.79,1174219.0,1.14482807
9,OTHER,23537.78,248721.0,10.56688439


In [118]:
conversion_ratio = mP_Summarised[['Summary_Portfolio', 'Uom', 'Value']].drop_duplicates(subset=['Summary_Portfolio', 'Uom']).pivot(
                                    index=['Summary_Portfolio', ],
                                    columns='Uom')['Value'].reset_index()

conversion_ratio['ZRW_ratio'] = conversion_ratio['ZRW'] / conversion_ratio['ZUC']
conversion_ratio['Trucks_ratio'] = conversion_ratio['Trucks'] / conversion_ratio['ZUC']
conversion_ratio['PAL_ratio'] = conversion_ratio['PAL'] / conversion_ratio['ZUC']
#x['GBP_ratio'] = x['GBP'] / x['ZUC']

conversion_ratio

Uom,Summary_Portfolio,GBP,PAL,Trucks,ZRW,ZUC,ZRW_ratio,Trucks_ratio,PAL_ratio
0,Aseptic,NaN,79.64,3.06,8601.40,9089.00,0.94635273,0.00033667,0.00876224
1,Cans - Board,NaN,59.77,2.30,7172.21,10004.00,0.71693423,0.00022991,0.00597461
2,Cans - Shrink,NaN,886.00,34.08,100850.39,140669.00,0.71693401,0.00024227,0.00629847
3,Cans - Singles,NaN,493.84,18.99,59260.33,82658.00,0.71693399,0.00022974,0.00597450
4,Capri Sun,NaN,1720.10,66.16,226632.39,198055.00,1.14429017,0.00033405,0.00868496
5,GB,0.12,0.01,0.00,0.48,1.00,0.48000000,0.00000000,0.01000000
6,Glass,NaN,187.53,7.21,46508.52,19658.00,2.36588259,0.00036677,0.00953963
7,LPET,NaN,0.00,0.00,0.20,0.47,0.42553191,0.00000000,0.00000000
8,Monster,NaN,3801.17,146.20,671962.50,699935.00,0.96003557,0.00020888,0.00543075
9,NL,NaN,173.86,6.69,27817.10,29394.00,0.94635300,0.00022760,0.00591481


In [18]:
# dP_siteSummarised = pd.read_pickle(os.path.join(pickle_dir, 'dP_siteSummarised'))
# dP_siteSummarised = dP_siteSummarised.drop_duplicates(subset=['Level2Region','Site_SAP','Year_Period','Summary_Portfolio',
#                                           'KeyField', 'Value', 'Uom'])
dP_siteSummarised.to_pickle(os.path.join(pickle_dir, 'dP_siteSummarised'))

In [67]:
SERVER_NAME = 'CVAPWPEQR001'
DATABASE_NAME = 'PACSQL_MTP'
DEMAND_TABLE = 'dbo.temp_dP_siteSummarised_materiallevel'
MATERIAL_HEADER_TABLE = 'material.Model_MaterialHeader_02'

import pyodbc

def fill_na_by_type(df, str_na='', float_na=0, int_na=0, bool_na=False):
    float_cols = df.select_dtypes(include=['float64']).columns
    str_cols = df.select_dtypes(include=['object']).columns

    df.loc[:, float_cols] = df.loc[:, float_cols].fillna(float_na)
    df.loc[:, str_cols] = df.loc[:, str_cols].fillna(str_na)

    return df

def trim_strings_df(df):
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    return df


def get_datatable_from_db(table_name, where_string=''):
    print(f'Fetching data for {table_name}')
    try:
        conn = pyodbc.connect('Driver={SQL Server};'
                            f'Server={SERVER_NAME};'
                            f'Database={DATABASE_NAME};'
                            'Trusted_Connection=yes;')

        sql = f'SELECT * FROM [{DATABASE_NAME}].{table_name} ' + where_string

        data = pd.read_sql(sql,conn)

        data = trim_strings_df(data)
        data = fill_na_by_type(data)

        conn.close()
    except Exception as e:
        raise Exception(f"Database Error: Unable to fetch data for [{DATABASE_NAME}].{table_name} Exception {e}")

    return data

In [21]:
dp = get_datatable_from_db(table_name=DEMAND_TABLE)

Fetching data for dbo.temp_dP_siteSummarised_materiallevel


In [22]:
mh = get_datatable_from_db(table_name=MATERIAL_HEADER_TABLE)

Fetching data for material.Model_MaterialHeader_02


In [29]:
dp.Material = dp.Material.astype(str)

Level2Region          object
Site_SAP               int64
Year_Period           object
Summary_Portfolio     object
Material              object
KeyField              object
Value                float64
Uom                   object
dtype: object

In [46]:
dp.groupby(['Summary_Portfolio', 'Material', 'Year_Period']).sum().reset_index()

,Summary_Portfolio,Material,Year_Period,Site_SAP,Value
0,Aseptic,203380,2020_06,66236,25185.80728531
1,Aseptic,203380,2020_07,66236,19043.11734772
2,Aseptic,203380,2020_08,66236,19011.97570038
3,Aseptic,203380,2020_09,66236,23959.60492134
4,Aseptic,203380,2020_10,66236,18871.83828735
...,...,...,...,...,...
6882,Water,466486,2021_01,88048,8645.69984245
6883,Water,466486,2021_02,88048,10697.15586090
6884,Water,466486,2021_03,88048,14492.54413033
6885,Water,466486,2021_04,88048,11467.91163254


In [30]:
mh.dtypes

SNPPL01                    object
GROUP                      object
COUNTRY                    object
Material                   object
FLAVOR                     object
SHELFLIVE                  object
Portfolio_DP               object
BUn                        object
BaseMaterial               object
BaseMaterialDescription    object
MaterialDescription        object
Producthierarchy           object
Level1                     object
Level2                     object
Level3                     object
Level4                     object
Level5                     object
Level1_Description         object
Level2_Description         object
Level3_Description         object
Level4_Description         object
Level5_Description         object
CreationDate               object
dtype: object

In [31]:
pd.merge(dp, mh[['SNPPL01', 'Material', 'FLAVOR', 'MaterialDescription']], on='Material')

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,Material,KeyField,Value,Uom,SNPPL01,FLAVOR,MaterialDescription
0,North,5502,2020_06,Aseptic,203380,Available,5387.58739376,ZRW,APET COTT GLAC (CP),GLACEAU VW POWER C,500MLNRP X12 GLAC PWR-C
1,North,5502,2020_06,Aseptic,203380,Available,5693.00000000,ZUC,APET COTT GLAC (CP),GLACEAU VW POWER C,500MLNRP X12 GLAC PWR-C
2,North,5502,2020_06,Aseptic,203380,Total,5387.58739376,ZRW,APET COTT GLAC (CP),GLACEAU VW POWER C,500MLNRP X12 GLAC PWR-C
3,North,5502,2020_06,Aseptic,203380,Total,5693.00000000,ZUC,APET COTT GLAC (CP),GLACEAU VW POWER C,500MLNRP X12 GLAC PWR-C
4,North,5502,2020_07,Aseptic,203380,Available,4074.04948711,ZRW,APET COTT GLAC (CP),GLACEAU VW POWER C,500MLNRP X12 GLAC PWR-C
...,...,...,...,...,...,...,...,...,...,...,...
103395,South,5560,2021_04,Cans - Board,264092,Total,633.00000000,ZUC,MBEV (CP),THUMS UP,300MLCAN X24 THUMS UP
103396,South,5560,2021_05,Cans - Board,264092,Available,124.60313642,ZRW,MBEV (CP),THUMS UP,300MLCAN X24 THUMS UP
103397,South,5560,2021_05,Cans - Board,264092,Available,158.00000000,ZUC,MBEV (CP),THUMS UP,300MLCAN X24 THUMS UP
103398,South,5560,2021_05,Cans - Board,264092,Total,124.60313642,ZRW,MBEV (CP),THUMS UP,300MLCAN X24 THUMS UP


In [17]:
material_list = '203380'
dp = dp.loc[(dp.Uom=='ZUC') & (dp.KeyField=='Total') & (dp.Year_Period=='2020_06')]
dp.loc[dp.Material.astype(str).isin([material_list])]

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,Material,KeyField,Value,Uom
3,North,5502,2020_06,Aseptic,203380,Total,5693.0,ZUC
48015,South,5501,2020_06,Aseptic,203380,Total,453.0,ZUC
96759,South,5556,2020_06,Aseptic,203380,Total,324.0,ZUC


In [3]:
dP_siteSummarised = pd.read_pickle(os.path.join(pickle_dir, 'dP_siteSummarised'))

demand_plan = dP_siteSummarised.loc[dP_siteSummarised.Uom == 'ZUC'].pivot(index=['Level2Region','Site_SAP','Year_Period','Summary_Portfolio','Material', 'Uom'], \
                                      columns='KeyField')['Value'].reset_index()

demand_plan['short_fall'] = demand_plan['Available'] - demand_plan['Total']
demand_plan['short_fall_pct'] = round((demand_plan['Available'] - demand_plan['Total'])*100/ demand_plan['Available'])

In [4]:
len(dP_siteSummarised.Material.unique())

9

In [23]:
demand_plan.loc[demand_plan.short_fall < 0] 

KeyField,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,Material,Uom,Available,Total,short_fall,short_fall_pct
1,North,5502,2020_06,Cans - Board,256816,ZUC,4.27076960e+06,4270771.0,-1.4000000,-0.0
6,North,5502,2020_06,Glass,246904,ZUC,4.14358000e+05,414360.0,-2.0000000,-0.0
12,North,5502,2020_06,REPACK,435648,ZUC,1.38990000e+04,29096.0,-15197.0000000,-109.0
32,North,5502,2020_07,Small Cans,236148,ZUC,3.72254722e+05,372255.0,-0.2780000,-0.0
35,North,5502,2020_08,Cans - Board,256816,ZUC,3.83194440e+06,3831945.0,-0.6000000,-0.0
...,...,...,...,...,...,...,...,...,...,...
904,South,5506,2020_12,Cans - Singles,407477,ZUC,2.75027333e+05,275028.0,-0.6665096,-0.0
907,South,5506,2020_12,LPET,427937,ZUC,6.87688200e+06,6876883.0,-1.0000000,-0.0
920,South,5506,2021_01,Cans - Singles,407477,ZUC,1.25963333e+05,125964.0,-0.6665096,-0.0
952,South,5506,2021_03,Cans - Singles,407477,ZUC,2.46967669e+05,246968.0,-0.3307818,-0.0


In [26]:
line_loading = pd.read_pickle('../input/pickle/iP_Summarised')
line_loading

,Table,Level2Region,Site_SAP,SNPPL01,KeyField,Value,Uom,SiteCountry,Summary_Portfolio,Material,Line_Portfolio,Year_Period
0,Inventory_MaterialLevel,,AS&D,REPACK (RP),GBP_ClosingCostOfCapital_Sum,0.0,GBP,GB,REPACK,435648,REPACK,2020_10
1,Inventory_MaterialLevel,,AS&D,REPACK (RP),GBP_ClosingStockValue_Avg,0.0,GBP,GB,REPACK,435648,REPACK,2020_10
2,Inventory_MaterialLevel,,AS&D,REPACK (RP),GBP_CostOfCapital_Sum,0.0,GBP,GB,REPACK,435648,REPACK,2020_10
3,Inventory_MaterialLevel,,AS&D,REPACK (RP),GBP_StockValue_Avg,0.0,GBP,GB,REPACK,435648,REPACK,2020_10
4,Inventory_MaterialLevel,,AS&D,REPACK (RP),PAL_ClosingStock,0.0,PAL,GB,REPACK,435648,REPACK,2020_10
...,...,...,...,...,...,...,...,...,...,...,...,...
436251,Inventory_MaterialLevel,South,5560,REPACK (RP),ZUC_StockVsTarget,0.0,ZUC,GB,REPACK,230116,REPACK,2020_12
436252,Inventory_MaterialLevel,South,5560,REPACK (RP),ZUC_StockVsTarget,24781.0,ZUC,GB,REPACK,230115,REPACK,2020_11
436253,Inventory_MaterialLevel,South,5560,REPACK (RP),ZUC_Stock_Avg,17221.0,ZUC,GB,REPACK,230116,REPACK,2020_09
436254,Inventory_MaterialLevel,South,5560,REPACK (RP),ZUC_Stock_Avg,20781.0,ZUC,GB,REPACK,230116,REPACK,2020_12


In [ ]:
# View Lines where Aseptic is manufacurted

In [ ]:
# Best place to manufacture product

In [120]:
# View Shift Lever

shift_lever = pd.read_csv('../input/levers/shift_lever.csv')

In [181]:
shift_lever.loc[(shift_lever.Site_SAP == 5501) 
                & (shift_lever.Line.isin(['LN03']))
                & (shift_lever.Period == '2020_06')
               ]

,Site_SAP,Line,Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,LossFactor,Hours_DownTime,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Delta_Delta
0,5501,LN03,2020_06,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0


In [134]:
delta_delta = 10

In [183]:
line_loading_plan.loc[(line_loading_plan.Line_Portfolio == 'Aseptic') \
                     &(line_loading_plan.Year_Period == '2020_06')
                     ]

,Line_Portfolio,Site_SAP,Line,Year_Period,KeyField,Value,Uom
970,Aseptic,5501,LN03,2020_06,Capacity,503.66,Hours
984,Aseptic,5501,LN03,2020_06,Consumed,503.66,Hours
1054,Aseptic,5501,LN03,2020_06,SpareHours,15.58,Hours


In [236]:
# Update Capacity and Spare Hours in Line Loading

line_loading_plan = pd.read_pickle(os.path.join(pickle_dir, 'sP_portSummarisedIHLineLoading')).round(2)
line_loading_plan = line_loading_plan.loc[line_loading_plan.Uom == 'Hours']

conditions = ((line_loading_plan.Site_SAP == 5501) \
            &(line_loading_plan.Line == 'LN03') \
            &(line_loading_plan.Year_Period == '2020_06') \
            &(line_loading_plan.KeyField.isin(['Capacity', 'SpareHours'])) \
            &(line_loading_plan.Line_Portfolio == 'Aseptic')
             )

line_loading_plan.loc[conditions, 'Value'] = line_loading_plan.loc[conditions]['Value'] + delta_delta

#line_loading_plan.loc[conditions, 'Value']
line_loading_plan.loc[(line_loading_plan.Line_Portfolio == 'Aseptic') \
                     &(line_loading_plan.Year_Period == '2020_06')
                     ]



,Line_Portfolio,Site_SAP,Line,Year_Period,KeyField,Value,Uom,New_Value
970,Aseptic,5501,LN03,2020_06,Capacity,503.66,Hours,513.66
984,Aseptic,5501,LN03,2020_06,Consumed,503.66,Hours,503.66
1054,Aseptic,5501,LN03,2020_06,SpareHours,15.58,Hours,25.58


In [188]:
# line_loading_plan = line_loading_plan.loc[line_loading_plan.Uom == 'Hours'].pivot(
#                             index=['Site_SAP','Line', 'Year_Period','Line_Portfolio', 'Uom'], 
#                             columns='KeyField')['Value'].reset_index()

# line_loading_plan

In [193]:
# Get Runrate
portfolio_runrate.loc[(portfolio_runrate.Plant == 5501) \
                      & (portfolio_runrate.Summary_Portfolio == 'Aseptic') \
                      & (portfolio_runrate.Line == 'LN03')
                     ]

,Plant,Line,Summary_Portfolio,BaseUnitsPerHour
6,5501,LN03,Aseptic,2380.34


In [223]:
#Time Required to manufacture shortfall

units_reqd = 10000

this_runrate = portfolio_runrate.loc[(portfolio_runrate.Plant == 5501) \
                      & (portfolio_runrate.Summary_Portfolio == 'Aseptic') \
                      & (portfolio_runrate.Line == 'LN03')
                     ]['BaseUnitsPerHour']

#this_runrate['required_units'] = units_reqd
#this_runrate['time_required_hours'] = round(this_runrate['required_units'] / this_runrate['BaseUnitsPerHour'], 2)

time_required_hours = float(round(units_reqd / this_runrate, 2))

time_required_hours

4.2

In [ ]:

# Check if spare hours are enough
if time_required_hours < delta_delta else delta_delta:
    time_to_use = time_required_hours 
else:
    time_to_use = delta_delta
    time_required_hours = time_required_hours - delta_delta


# Update line_loading_plan
conditions_consumed = ((line_loading_plan.Site_SAP == 5501) \
            &(line_loading_plan.Line == 'LN03') \
            &(line_loading_plan.Year_Period == '2020_06') \
            &(line_loading_plan.KeyField.isin(['Consumed'])) \
            &(line_loading_plan.Line_Portfolio == 'Aseptic')
             )

conditions_spare = ((line_loading_plan.Site_SAP == 5501) \
            &(line_loading_plan.Line == 'LN03') \
            &(line_loading_plan.Year_Period == '2020_06') \
            &(line_loading_plan.KeyField.isin(['SpareHours'])) \
            &(line_loading_plan.Line_Portfolio == 'Aseptic')
             )

line_loading_plan.loc[conditions_consumed, 'Value'] = line_loading_plan.loc[conditions_consumed]['Value'] + time_to_use

line_loading_plan.loc[conditions_spare, 'Value'] = line_loading_plan.loc[conditions_spare]['Value'] - time_to_use


## ***** if demand site not equal to prod site

# Update dp

#units manufactured in time_to_use
units_made = float(this_runrate * time_to_use)

conditions_demand = (dP_siteSummarised.Site_SAP == 5502) \
            &(dP_siteSummarised.Level2Region == 'North') \
            &(dP_siteSummarised.Year_Period == '2020_06') \
            &(dP_siteSummarised.KeyField.isin(['Available']) \
            &(dP_siteSummarised.Summary_Portfolio == 'Aseptic')
             )

dP_siteSummarised.loc[conditions_demand, 'Value'] = dP_siteSummarised.loc[conditions_demand, 'Value']  + units_made


# update mp
# if demand site not equal to prod site
# Update MP
mP_Summarised = pd.read_pickle(os.path.join(pickle_dir, 'mP_Summarised')).round(2)


conditions_movement = (mP_Summarised.Site_SAP_From == 5501) \
            &(mP_Summarised.Site_SAP_To == 5502) \
            &(mP_Summarised.Year_Period == '2020_06') \
            &(mP_Summarised.KeyField.isin(['ZUC']) \
            &(mP_Summarised.Summary_Portfolio == 'Aseptic')
             )

mP_Summarised[conditions_movement,'Value'] = mP_Summarised[conditions_movement,'Value'] + units_made


,Plant,Line,Summary_Portfolio,BaseUnitsPerHour
0,5501,LN01,Cans - Shrink,6433.33
1,5501,LN01,Cans - Singles,5013.44
2,5501,LN01,Monster,4961.53
3,5501,LN01,Relentless,5266.64
4,5501,LN02,REPACK,2177.74
...,...,...,...,...
196,5559,LN01,REPACK,9012.83
197,5560,LN01,Cans - Board,10000.00
198,5560,LN01,REPACK,7919.78
199,5567,LN01,Cans - Board,10000.00


In [192]:
line_loading_plan.loc[(line_loading_plan.Line_Portfolio == 'Aseptic') \
                     &(line_loading_plan.Year_Period == '2020_06')
                     ]

KeyField,Site_SAP,Line,Year_Period,Line_Portfolio,Uom,Capacity,Consumed,SpareHours
22,5501,LN03,2020_06,Aseptic,Hours,513.66,503.66,25.58


In [199]:
line_loading_plan.loc[(line_loading_plan.Line_Portfolio == 'Aseptic') \
                     &(line_loading_plan.Year_Period == '2020_06')
                     ]

,Line_Portfolio,Site_SAP,Line,Year_Period,KeyField,Value,Uom
970,Aseptic,5501,LN03,2020_06,Capacity,513.66,Hours
984,Aseptic,5501,LN03,2020_06,Consumed,503.66,Hours
1054,Aseptic,5501,LN03,2020_06,SpareHours,25.58,Hours


In [231]:
line_loading_plan.loc[(line_loading_plan.Line_Portfolio == 'Aseptic') \
                     &(line_loading_plan.Year_Period == '2020_06')
                     ]

,Line_Portfolio,Site_SAP,Line,Year_Period,KeyField,Value,Uom
970,Aseptic,5501,LN03,2020_06,Capacity,513.66,Hours
984,Aseptic,5501,LN03,2020_06,Consumed,507.86,Hours
1054,Aseptic,5501,LN03,2020_06,SpareHours,21.38,Hours


In [163]:
def update_line_loading(site, line, period, portfolio, pickle_dir, additional_hours=None, used_hours=None):
    
    line_loading_plan = pd.read_pickle(os.path.join(pickle_dir, 'sP_portSummarisedIHLineLoading')).round(2)    
    line_loading_plan = line_loading_plan.loc[line_loading_plan.Uom == 'Hours']
    
    base_condition = ((line_loading_plan.Site_SAP == site) \
                      & (line_loading_plan.Line == line) \
                      & (line_loading_plan.Year_Period == period) \
                      & (line_loading_plan.Line_Portfolio == portfolio) \
                     )

    if used_hours is None:
        # add hours to consumed and capacity
        condition = base_condition & (line_loading_plan.KeyField.isin(['Capacity', 'SpareHours']))

        line_loading_plan.loc[condition, 'Value'] = line_loading_plan.loc[condition]['Value'] + additional_hours    
        
    elif additional_hours is None:
        # Update line_loading_plan with used time to consumed and spare hours
        conditions_consumed = base_condition & (line_loading_plan.KeyField.isin(['Consumed']))         
        conditions_spare = base_condition &(line_loading_plan.KeyField.isin(['SpareHours']))

        line_loading_plan.loc[conditions_consumed, 'Value'] = line_loading_plan.loc[conditions_consumed]['Value'] + used_hours

        line_loading_plan.loc[conditions_spare, 'Value'] = line_loading_plan.loc[conditions_spare]['Value'] - used_hours
        
    
    
    line_loading_plan.to_pickle(os.path.join(pickle_dir, 'sP_portSummarisedIHLineLoading'))
    
    
def get_time_required(site, line, portfolio, metadata_dir):
    
    # Create Runrate matrix
    material_snp_mapping = pd.read_csv(os.path.join(metadata_dir, 'material_snp_mapping.csv'))
    runrate = pd.read_csv(os.path.join(metadata_dir, 'runrate.csv'))
    snp_portfolio_mapping = pd.read_csv(os.path.join(metadata_dir, 'snp_portfolio_mapping.csv'))

    material_runrate = pd.merge(pd.merge(material_snp_mapping, snp_portfolio_mapping, on='SNPPL01', how='inner'),
                                runrate[['Material', 'BaseUnitsPerHour', 'Plant', 'Line']], on='Material', how='inner')
    portfolio_runrate = material_runrate[['Plant', 'Line','Summary_Portfolio', 'BaseUnitsPerHour']]\
                        .groupby(['Plant', 'Line','Summary_Portfolio']).mean().round(2).reset_index()
    
    
    #Time Required to manufacture shortfall    

    this_runrate = portfolio_runrate.loc[(portfolio_runrate.Plant == site) \
                          & (portfolio_runrate.Summary_Portfolio == portfolio) \
                          & (portfolio_runrate.Line == line)
                         ]['BaseUnitsPerHour']

    time_required_hours = float(round(additional_units_required / this_runrate, 2))
    
    return time_required_hours, this_runrate


def update_demand_movement_plan(demand_site, supply_site, period, portfolio, units_made, 
                                pickle_dir, conversion_table):  
    
    # Update Demand      
    try:
        dP_siteSummarised = pd.read_pickle(os.path.join(pickle_dir, 'dP_siteSummarised'))

        conditions_demand = ( (dP_siteSummarised.Site_SAP == demand_site) \
                             &(dP_siteSummarised.Year_Period == period) \
                             &(dP_siteSummarised.KeyField=='Available') & (dP_siteSummarised.Summary_Portfolio == portfolio)
                            )
                
        for uom in list(dP_siteSummarised.Uom.unique()):            
            conversion_ratio=float(conversion_table.loc[conversion_table.Summary_Portfolio==portfolio, uom+'_ratio'])
            units = units_made*conversion_ratio                         
            condition = conditions_demand & (dP_siteSummarised.Uom == uom)
            dP_siteSummarised.loc[condition, 'Value'] = dP_siteSummarised.loc[condition, 'Value']  + units

        dP_siteSummarised.to_pickle(os.path.join(pickle_dir, 'dP_siteSummarised'))
                                                
    except Exception as e:
        print(f"Exception while updating Demand Plan {e}")
    
    if demand_site != supply_site:
        # update Movement Plan
        try:
            mP_Summarised = pd.read_pickle(os.path.join(pickle_dir, 'mP_Summarised')).round(2)

            conditions_movement = ((mP_Summarised.Site_SAP_From == str(supply_site)) & (mP_Summarised.Site_SAP_To == str(demand_site)) & (mP_Summarised.Year_Period == period)  & (mP_Summarised.Summary_Portfolio == portfolio))
            
            uom_list = list(mP_Summarised.Uom.unique())
            uom_list.remove('GBP')
            for uom in uom_list:        
                conversion_ratio=float(conversion_table.loc[conversion_table.Summary_Portfolio==portfolio,uom+'_ratio'])
                
                units = units_made*conversion_ratio                         
                condition = conditions_movement & (mP_Summarised.KeyField == uom)
                
                print(f"ratio: {conversion_ratio} old value {mP_Summarised.loc[condition, 'Value'] } new value { mP_Summarised.loc[condition, 'Value']  + units} ")
                
                mP_Summarised.loc[condition, 'Value'] = mP_Summarised.loc[condition, 'Value']  + units

            mP_Summarised.loc[conditions_movement,'Value'] = mP_Summarised.loc[conditions_movement,'Value'] + units_made

            mP_Summarised.to_pickle(os.path.join(pickle_dir, 'mP_Summarised'))
        except Exception as e:
            print(f"Exception while updating Movement Plan {e}")


In [164]:
supply_site=5501
supply_line='LN03'
period = '2020_06'
portfolio = 'Aseptic'
demand_site=5502
additional_hours_added = 10
additional_units_required = 10000
pickle_dir = '../output/pickle'
metadata_dir = '../input/metadata'
csv_dir = '../input/csv'

conversion_table = mP_Summarised[['Summary_Portfolio', 'Uom', 'Value']].drop_duplicates(subset=['Summary_Portfolio', 'Uom']).pivot(
                                    index=['Summary_Portfolio', ],
                                    columns='Uom')['Value'].reset_index()

conversion_table['ZUC_ratio'] = conversion_table['ZUC'] / conversion_table['ZUC']
conversion_table['ZRW_ratio'] = conversion_table['ZRW'] / conversion_table['ZUC']
conversion_table['Trucks_ratio'] = conversion_table['Trucks'] / conversion_table['ZUC']
conversion_table['PAL_ratio'] = conversion_table['PAL'] / conversion_table['ZUC']
#x['GBP_ratio'] = x['GBP'] / x['ZUC']


def update_plans(additional_hours_added, additional_units_required, demand_site,
                 portfolio, supply_site, supply_line, period, pickle_dir, metadata_dir,
                 conversion_table):
    #' Update Line Loading with additional hours to capacity and spare
    update_line_loading(supply_site, supply_line, period, portfolio, pickle_dir,
                        additional_hours=additional_hours_added)    
    
    #Time Required to manufacture shortfall    
    time_required_hours, runrate = get_time_required(supply_site, supply_line, portfolio, metadata_dir)
    
    
    # Check if additional hours are enough
    if time_required_hours < additional_hours_added:
        time_to_use = time_required_hours 
    else:
        time_to_use = additional_hours_added
        time_required_hours = time_required_hours - additional_hours_added

    # Update line_loading_plan with used time to consumed and spare hours
    update_line_loading(supply_site, supply_line, period, portfolio,  pickle_dir,
                        used_hours=time_to_use)       
    
    # Update Demand / Movement plans    
    units_made = float(runrate * time_to_use)
    
    update_demand_movement_plan(demand_site, supply_site, period, portfolio, units_made, pickle_dir, conversion_table)

    
    
    


In [165]:
conversion_table.head()

Uom,Summary_Portfolio,GBP,PAL,Trucks,ZRW,ZUC,ZUC_ratio,ZRW_ratio,Trucks_ratio,PAL_ratio
0,Aseptic,NaN,79.64,3.06,8601.40,9089.0,1.0,0.94635273,0.00033667,0.00876224
1,Cans - Board,NaN,59.77,2.30,7172.21,10004.0,1.0,0.71693423,0.00022991,0.00597461
2,Cans - Shrink,NaN,886.00,34.08,100850.39,140669.0,1.0,0.71693401,0.00024227,0.00629847
3,Cans - Singles,NaN,493.84,18.99,59260.33,82658.0,1.0,0.71693399,0.00022974,0.00597450
4,Capri Sun,NaN,1720.10,66.16,226632.39,198055.0,1.0,1.14429017,0.00033405,0.00868496


In [166]:
update_plans(additional_hours_added, additional_units_required, demand_site,
            portfolio, supply_site, supply_line, period, pickle_dir, metadata_dir,
            conversion_table)

ratio: 0.008762240070414788 old value 11396    20313.84
Name: Value, dtype: float64 new value 11396    20401.43986422
Name: Value, dtype: float64 
ratio: 0.0003366707008471779 old value 11397    20007.11
Name: Value, dtype: float64 new value 11397    20010.47584109
Name: Value, dtype: float64 
ratio: 0.9463527340741555 old value 11398    59621.35
Name: Value, dtype: float64 new value 11398    69082.44332151
Name: Value, dtype: float64 
ratio: 1.0 old value 11399    71865.15
Name: Value, dtype: float64 new value 11399    81862.578
Name: Value, dtype: float64 


In [149]:
input_demand = pd.read_csv(os.path.join(csv_dir, 'dP_siteSummarised.csv')).round(2)
output_demand = pd.read_pickle(os.path.join(pickle_dir, 'dP_siteSummarised')).round(2)

In [150]:
input_demand.head()

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,KeyField,Value,Uom
0,North,5502,2020_06,Aseptic,Available,79015.33,ZRW
1,North,5502,2020_06,Aseptic,Available,84895.00,ZUC
2,North,5502,2020_06,Aseptic,Total,79015.33,ZRW
3,North,5502,2020_06,Aseptic,Total,84895.00,ZUC
4,North,5502,2020_06,Cans - Board,Available,2654606.00,ZRW


In [151]:
output_demand.head()

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,KeyField,Value,Uom
0,North,5502,2020_06,Aseptic,Available,88476.42,ZRW
1,North,5502,2020_06,Aseptic,Available,114884.71,ZUC
2,North,5502,2020_06,Aseptic,Total,79015.33,ZRW
3,North,5502,2020_06,Aseptic,Total,84895.00,ZUC
4,North,5502,2020_06,Cans - Board,Available,2654606.00,ZRW


In [153]:
input_movement = pd.read_csv(os.path.join(csv_dir, 'mP_Summarised.csv')).round(2)
output_movement = pd.read_pickle(os.path.join(pickle_dir, 'mP_Summarised')).round(2)

In [158]:
input_movement.loc[(input_movement.Summary_Portfolio=='Aseptic') 
                   & (input_movement.Year_Period=='2020_06')
                   & (input_movement.Site_SAP_To=='5502')
                   & (input_movement.Site_SAP_From=='5501')
                  ]

,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,KeyField,Value,Uom,ToRepack_Flag
11395,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,GBP_MovementCost,2637.26,GBP,False
11396,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,PAL,143.78,PAL,False
11397,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,Trucks,5.53,Trucks,False
11398,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,ZRW,20704.31,ZRW,False
11399,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,ZUC,21878.00,ZUC,False


In [159]:
output_movement.loc[(output_movement.Summary_Portfolio=='Aseptic') 
                    & (output_movement.Year_Period=='2020_06') 
                    & (output_movement.Site_SAP_To=='5502')
                    & (output_movement.Site_SAP_From=='5501')
                   ]

,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,KeyField,Value,Uom,ToRepack_Flag
11395,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,GBP_MovementCost,12634.69,GBP,False
11396,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,PAL,10228.81,PAL,False
11397,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,Trucks,10006.32,Trucks,False
11398,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,ZRW,40162.83,ZRW,False
11399,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,ZUC,51870.29,ZUC,False


In [79]:
conditions_movement = ((mP_Summarised.Site_SAP_From == str(supply_site)) \
                        &(mP_Summarised.Site_SAP_To == str(demand_site)) \
                        &(mP_Summarised.Year_Period == period) \
                        &(mP_Summarised.KeyField == 'ZUC') \
                        &(mP_Summarised.Summary_Portfolio == portfolio)
                    )

mP_Summarised[conditions_movement].head()            

,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,KeyField,Value,Uom,ToRepack_Flag
11399,GB,GB,NaN,South,North,5501,5502,2020_06,Aseptic,ZUC,21878.0,ZUC,False


In [128]:
uom_list = list(mP_Summarised.Uom.unique())
uom_list.remove('GBP')
for x in uom_list:
    print(x)

GBP
PAL
Trucks
ZRW
ZUC


In [127]:
conversion_table = mP_Summarised[['Summary_Portfolio', 'Uom', 'Value']].drop_duplicates(subset=['Summary_Portfolio', 'Uom']).pivot(
                                    index=['Summary_Portfolio', ],
                                    columns='Uom')['Value'].reset_index()

conversion_table['ZUC_ratio'] = conversion_table['ZUC'] / conversion_table['ZUC']
conversion_table['ZRW_ratio'] = conversion_table['ZRW'] / conversion_table['ZUC']
conversion_table['Trucks_ratio'] = conversion_table['Trucks'] / conversion_table['ZUC']
conversion_table['PAL_ratio'] = conversion_table['PAL'] / conversion_table['ZUC']

#x['GBP_ratio'] = x['GBP'] / x['ZUC']

conversion_table

Uom,Summary_Portfolio,GBP,PAL,Trucks,ZRW,ZUC,ZUC_ratio,ZRW_ratio,Trucks_ratio,PAL_ratio
0,Aseptic,NaN,79.64,3.06,8601.40,9089.00,1.0,0.94635273,0.00033667,0.00876224
1,Cans - Board,NaN,59.77,2.30,7172.21,10004.00,1.0,0.71693423,0.00022991,0.00597461
2,Cans - Shrink,NaN,886.00,34.08,100850.39,140669.00,1.0,0.71693401,0.00024227,0.00629847
3,Cans - Singles,NaN,493.84,18.99,59260.33,82658.00,1.0,0.71693399,0.00022974,0.00597450
4,Capri Sun,NaN,1720.10,66.16,226632.39,198055.00,1.0,1.14429017,0.00033405,0.00868496
5,GB,0.12,0.01,0.00,0.48,1.00,1.0,0.48000000,0.00000000,0.01000000
6,Glass,NaN,187.53,7.21,46508.52,19658.00,1.0,2.36588259,0.00036677,0.00953963
7,LPET,NaN,0.00,0.00,0.20,0.47,1.0,0.42553191,0.00000000,0.00000000
8,Monster,NaN,3801.17,146.20,671962.50,699935.00,1.0,0.96003557,0.00020888,0.00543075
9,NL,NaN,173.86,6.69,27817.10,29394.00,1.0,0.94635300,0.00022760,0.00591481


In [20]:
shift_lever = pd.read_csv('../input/levers/shift_lever.csv')
line_loading = pd.read_csv('../input/csv/sP_portSummarisedIHLineLoading.csv')
line_loading = line_loading[['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period']].drop_duplicates()
line_loading.columns = ['Portfolio', 'Site_SAP', 'Line', 'Period']


pd.merge(shift_lever, line_loading, on=['Site_SAP', 'Line', 'Period'])

,Site_SAP,Line,Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,LossFactor,Hours_DownTime,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Additional_Hours,Portfolio
0,5501,LN03,2020_06,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0,Aseptic
1,5501,LN03,2020_06,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0,Oasis
2,5501,LN03,2020_07,4,8,3,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,Aseptic
3,5501,LN03,2020_07,4,8,3,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,Oasis
4,5501,LN03,2020_08,4,24,1,5,0,672,0,0.25,34,159.50,0.97,464.1450,0,Aseptic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,5506,LN02,2020_12,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,LPET
300,5506,LN02,2021_01,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,LPET
301,5506,LN02,2021_02,5,24,1,7,0,840,0,0.25,363,119.25,0.97,347.0175,0,LPET
302,5506,LN02,2021_03,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0,LPET


In [21]:
csv_dir = '../input/csv'

dP_siteSummarised = pd.read_csv(os.path.join(csv_dir, 'dP_siteSummarised.csv')).round(2)

# Demand Shortfalls
demand_plan = dP_siteSummarised.loc[dP_siteSummarised.Uom == 'ZUC'].pivot(index=['Level2Region','Site_SAP','Year_Period','Summary_Portfolio', 'Uom'], \
                                      columns='KeyField')['Value'].reset_index()

demand_plan['short_fall'] = demand_plan['Available'] - demand_plan['Total']
demand_plan['short_fall_pct'] = round((demand_plan['Available'] - demand_plan['Total'])*100/ demand_plan['Available'])

short_falls = demand_plan.loc[demand_plan.short_fall_pct < -10] 

short_falls = short_falls[['Site_SAP', 'Summary_Portfolio', 'Year_Period', 'short_fall']]

short_falls.columns = ['Demand_Site', 'Portfolio', 'Period', 'Additional_Units_Required']

short_falls

,Demand_Site,Portfolio,Period,Additional_Units_Required
0,5502,Aseptic,2020_06,-10000.0
12,5502,REPACK,2020_06,-15197.0
357,5567,REPACK,2020_06,-26277.0
372,5569,REPACK,2020_06,-26277.0
424,5503,REPACK,2020_06,-1787.0
816,5506,REPACK,2020_06,-12810.0


In [22]:
shift_lever = pd.merge(shift_lever, line_loading, on=['Site_SAP', 'Line', 'Period'])
shift_lever = pd.merge(shift_lever, short_falls, on=['Period', 'Portfolio'], how='left')

shift_lever

,Site_SAP,Line,Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,LossFactor,Hours_DownTime,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Additional_Hours,Portfolio,Demand_Site,Additional_Units_Required
0,5501,LN03,2020_06,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0,Aseptic,5502.0,-10000.0
1,5501,LN03,2020_06,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0,Oasis,NaN,NaN
2,5501,LN03,2020_07,4,8,3,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,Aseptic,NaN,NaN
3,5501,LN03,2020_07,4,8,3,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,Oasis,NaN,NaN
4,5501,LN03,2020_08,4,24,1,5,0,672,0,0.25,34,159.50,0.97,464.1450,0,Aseptic,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,5506,LN02,2020_12,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,LPET,NaN,NaN
300,5506,LN02,2021_01,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,LPET,NaN,NaN
301,5506,LN02,2021_02,5,24,1,7,0,840,0,0.25,363,119.25,0.97,347.0175,0,LPET,NaN,NaN
302,5506,LN02,2021_03,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0,LPET,NaN,NaN


# MEETING STOCK TARGET

In [3]:
iP_Summarised = pd.read_csv(os.path.join(csv_dir, 'iP_Summarised.csv')).round(2)

iP_Summarised.head()

,Table,Level2Region,Site_SAP,SNPPL01,KeyField,Value,Uom,SiteCountry,Summary_Portfolio,Line_Portfolio,Year_Period
0,Inventory,NaN,NaN,NaN,Days_ClosingStockTarget_National,NaN,Days,GB,NaN,NaN,2021_05
1,Inventory,NaN,NaN,NaN,Days_ClosingStockTarget_National,NaN,Days,GB,NaN,NaN,2021_06
2,Inventory,NaN,NaN,NaN,Days_ClosingStockTarget_National,NaN,Days,GB,NaN,NaN,2021_07
3,Inventory,NaN,NaN,NaN,Days_ClosingStockTarget_National,NaN,Days,GB,NaN,NaN,2021_08
4,Inventory,NaN,NaN,NaN,Days_ClosingStockTarget_National,17.45,Days,GB,NaN,NaN,2021_01


In [4]:
iP_Summarised.KeyField.value_counts()

Year_Compare                                         4451
GBP_ClosingStockValue_Avg                            2198
GBP_ClosingCostOfCapital_Sum                         2198
ZUC_StockVsTarget                                    2198
ZRW_Stock_Avg                                        2198
GBP_CostOfCapital_Sum                                2198
ZUC_ClosingStockTarget                               2198
ZUC_ClosingStock                                     2198
ZUC_Stock_Avg                                        2198
PAL_ClosingStockTarget                               2198
ZRW_ClosingStock                                     2198
ZRW_ClosingStockTarget                               2198
GBP_StockValue_Avg                                   2198
PAL_ClosingStock                                     2198
PAL_StockVsTarget                                    2198
ZRW_StockVsTarget                                    2198
PAL_Stock_Avg                                        2198
Days_ClosingSt

In [12]:
pivot_cols = ['KeyField']
pivot_rows = ['Site_SAP', 'Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom': 'ZUC'}

iP_Summarised = pd.read_csv(os.path.join(csv_dir, 'iP_Summarised.csv')).round(2)

df = iP_Summarised[iP_Summarised.KeyField.isin(['ZUC_ClosingStockTarget', 'ZUC_StockVsTarget'])]

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)

wide_df = pd.DataFrame(wide_df.to_records())

wide_df.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in wide_df.columns]

wide_df['inventory_shortfall_pct'] = round(wide_df['ZUC_StockVsTarget']*-100 / wide_df['ZUC_ClosingStockTarget'], 2)

wide_df['inventory_shortfall'] = -1*wide_df['ZUC_StockVsTarget']

wide_df = wide_df[['Site_SAP', 'Summary_Portfolio', 'Year_Period', 'inventory_shortfall', 'inventory_shortfall_pct']]

wide_df.columns = ['Inventory_Site', 'Portfolio', 'Period', 'Inventory_shortfall', 'inventory_shortfall_pct']

wide_df

,Inventory_Site,Portfolio,Period,Inventory_shortfall,inventory_shortfall_pct
0,5501,Aseptic,2020_06,1891.00,4.88
1,5501,Aseptic,2020_07,3765.00,9.72
2,5501,Aseptic,2020_08,3183.71,8.50
3,5501,Aseptic,2020_09,-91759.86,-289.93
4,5501,Aseptic,2020_10,-16369.85,-54.36
...,...,...,...,...,...
1377,RWAR_RP,Cans - Singles,2020_06,-0.00,NaN
1378,RWAR_RP,Cans - Singles,2020_07,-0.00,NaN
1379,RWAR_RP,Cans - Singles,2020_09,-0.00,NaN
1380,RWAR_RP,Cans - Singles,2020_12,-0.00,NaN


In [14]:
INPUT_PICKLE_DIR = '../input/pickle'
mP_Summarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'mP_Summarised'))
mP_Summarised.loc[mP_Summarised.Material == '100691'] 

,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,Material,KeyField,Value,Uom,ToRepack_Flag
99790,GB,SE,,South,,5501,N11,2020_06,Aseptic,100691,GBP_MovementCost,0.0,GBP,False
99791,GB,SE,,South,,5501,N11,2020_06,Aseptic,100691,PAL,0.0,PAL,False
99792,GB,SE,,South,,5501,N11,2020_06,Aseptic,100691,Trucks,0.0,Trucks,False
99793,GB,SE,,South,,5501,N11,2020_06,Aseptic,100691,ZRW,0.0,ZRW,False
99794,GB,SE,,South,,5501,N11,2020_06,Aseptic,100691,ZUC,51129.0,ZUC,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100560,GB,SE,,South,,5501,N11,2021_07,Aseptic,100691,GBP_MovementCost,0.0,GBP,False
100561,GB,SE,,South,,5501,N11,2021_07,Aseptic,100691,PAL,0.0,PAL,False
100562,GB,SE,,South,,5501,N11,2021_07,Aseptic,100691,Trucks,0.0,Trucks,False
100563,GB,SE,,South,,5501,N11,2021_07,Aseptic,100691,ZRW,0.0,ZRW,False


In [12]:
conversion_table = mP_Summarised[['Material', 'Uom', 'Value']].drop_duplicates(subset=['Material', 'Uom']).pivot(
                                            index=['Material'],
                                            columns='Uom')['Value'].reset_index()

conversion_table['ZUC_ratio'] = conversion_table['ZUC'] / conversion_table['ZUC']
conversion_table['ZRW_ratio'] = conversion_table['ZRW'] / conversion_table['ZUC']
conversion_table['Trucks_ratio'] = conversion_table['Trucks'] / conversion_table['ZUC']
conversion_table['PAL_ratio'] = conversion_table['PAL'] / conversion_table['ZUC']
conversion_table['GBP_ratio'] = conversion_table['GBP'] / conversion_table['ZUC']

conversion_table.loc[conversion_table.ZRW_ratio == 0]

Uom,Material,GBP,PAL,Trucks,ZRW,ZUC,ZUC_ratio,ZRW_ratio,Trucks_ratio,PAL_ratio,GBP_ratio
0,090865,0.0,0.0,0.0,0.0,24894.0000000,1.0,0.0,0.0,0.0,0.0
1,100691,0.0,0.0,0.0,0.0,51129.0000000,1.0,0.0,0.0,0.0,0.0
2,101310,0.0,0.0,0.0,0.0,78814.0000000,1.0,0.0,0.0,0.0,0.0
3,101311,0.0,0.0,0.0,0.0,41924.0000000,1.0,0.0,0.0,0.0,0.0
4,101312,0.0,0.0,0.0,0.0,77583.0000000,1.0,0.0,0.0,0.0,0.0
5,102710,0.0,0.0,0.0,0.0,75876.6600000,1.0,0.0,0.0,0.0,0.0
6,104105,0.0,0.0,0.0,0.0,11034.0000000,1.0,0.0,0.0,0.0,0.0
7,104194,0.0,0.0,0.0,0.0,6193.0000000,1.0,0.0,0.0,0.0,0.0
8,104910,0.0,0.0,0.0,0.0,3105.0000000,1.0,0.0,0.0,0.0,0.0
9,106153,0.0,0.0,0.0,0.0,9977.0000000,1.0,0.0,0.0,0.0,0.0


In [6]:
line_loading = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'sP_portSummarisedIHLineLoading'))
line_loading

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,090865,5501,LN03,2020_06,Planned,0.0,ZRW
1,Aseptic,090865,5501,LN03,2020_06,Planned,24894.0,ZUC
2,Aseptic,090865,5501,LN03,2020_07,Planned,0.0,ZRW
3,Aseptic,090865,5501,LN03,2020_07,Planned,9842.0,ZUC
4,Aseptic,090865,5501,LN03,2020_08,Planned,0.0,ZRW
...,...,...,...,...,...,...,...,...
20783,Water,,5546,LN01,2021_02,SpareHours,0.0,Hours
20784,Water,,5546,LN01,2021_03,SpareHours,0.0,Hours
20785,Water,,5546,LN01,2021_04,SpareHours,0.0,Hours
20786,Water,,5546,LN01,2021_05,SpareHours,0.0,Hours


In [15]:
material_runrate = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'production_capacity'))
material_runrate

,Plant,ProductionVersion,Line,RecievingSLOC,ProductionVersionStatus,Material,MaterialDescription,ProcurementType,Group,ProductionTime,Unitofmeasureforthestandardvalue,BaseQuantity,UnitofMeasureforActivityOperation,ValidTo,ValidFrom,LastCheckDate_Out,BaseUnitsPerHour,DBLoadDate
0,5501,P010,LN01,SL01,1,216789,330MLCAN 6P COCA-COLA X4 UDP9L,X,50062817,0.635,H,10000.0,PAC,2019-12-23,2019-02-19,2019-12-23,15748.03149606,2020-10-07 14:03:19
1,5501,P010,LN01,SL01,1,277956,330MLCAN 8P DT COKE TPNK ASDA X3 UDP,X,50077068,0.941,H,10000.0,PAC,9999-12-31,2020-08-18,2020-08-18,10626.99256111,2020-10-07 14:03:19
2,5501,P010,LN01,SL01,1,407611,330MLCAN 8P COKE CLASSIC X3 MU 11H,X,50077097,0.941,H,10000.0,PAC,9999-12-31,2020-08-21,2020-08-21,10626.99256111,2020-10-07 14:03:19
3,5501,P010,LN01,SL01,1,407733,330MLCAN 8P DT COKE X3 MU 11H,X,50062818,0.941,H,10000.0,PAC,2019-12-23,2019-02-19,2019-12-23,10626.99256111,2020-10-07 14:03:19
4,5501,P010,LN01,SL01,1,465044,TRIAL PRODUCT 120 CS CHEP,X,50062991,2.016,H,10000.0,CS,9999-12-31,2019-03-01,2019-11-14,4960.31746032,2020-10-07 14:03:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9365,5567,P999,LN01,REPK,1,267378,1LNRP X384 SW VP SWT/SWSLT ALDI UDP,X,50076244,1.000,H,10000.0,PAL,9999-12-31,2020-06-15,2020-09-09,10000.00000000,2020-10-07 14:03:19
9366,5567,P999,LN01,REPK,1,271170,330MLCAN 8P SPNS VP SPNS50 SP50 X3 UDP,X,50076245,1.000,H,10000.0,PAC,9999-12-31,2020-06-15,2020-09-09,10000.00000000,2020-10-07 14:03:19
9367,5567,P999,LN01,REPK,1,272764,2.25LEPET X210 DT COKE VP DTC59 CZ41 UDP,X,50076246,1.000,H,10000.0,PAL,9999-12-31,2020-06-15,2020-09-09,10000.00000000,2020-10-07 14:03:19
9368,5567,P999,LN01,REPK,1,277002,150MLCAN 24P SW TONW50 SLTNIC50 ASDA UDP,X,50076473,1.000,H,10000.0,PAC,9999-12-31,2020-07-03,2020-07-03,10000.00000000,2020-10-07 14:03:19


In [31]:
material_runrate = material_runrate.drop_duplicates(subset=['Plant', 'Line', 'Material'])
material_runrate.loc[(material_runrate.Plant == '5502') \
                    & (material_runrate.Material == '201051') \
                    & (material_runrate.Line == 'LN01')
                    ]

,Plant,ProductionVersion,Line,RecievingSLOC,ProductionVersionStatus,Material,MaterialDescription,ProcurementType,Group,ProductionTime,Unitofmeasureforthestandardvalue,BaseQuantity,UnitofMeasureforActivityOperation,ValidTo,ValidFrom,LastCheckDate_Out,BaseUnitsPerHour,DBLoadDate


In [25]:
#Time Required to manufacture shortfall    

this_runrate = material_runrate.loc[(material_runrate.Plant == '5502') \
                    & (material_runrate.Material == '106513') \
                    & (material_runrate.Line == 'LN01')
                    ]['BaseUnitsPerHour']
this_runrate

Series([], Name: BaseUnitsPerHour, dtype: float64)

In [24]:
float(round(10000 / this_runrate, 4))

0.635

In [69]:
material_runrate = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'production_capacity'))
material_runrate

,Plant,ProductionVersion,Line,RecievingSLOC,ProductionVersionStatus,Material,MaterialDescription,ProcurementType,Group,ProductionTime,Unitofmeasureforthestandardvalue,BaseQuantity,UnitofMeasureforActivityOperation,ValidTo,ValidFrom,LastCheckDate_Out,BaseUnitsPerHour,DBLoadDate
0,5501,P010,LN01,SL01,1,216789,330MLCAN 6P COCA-COLA X4 UDP9L,X,50062817,0.635,H,10000.0,PAC,2019-12-23,2019-02-19,2019-12-23,15748.03149606,2020-10-07 14:03:19
1,5501,P010,LN01,SL01,1,277956,330MLCAN 8P DT COKE TPNK ASDA X3 UDP,X,50077068,0.941,H,10000.0,PAC,9999-12-31,2020-08-18,2020-08-18,10626.99256111,2020-10-07 14:03:19
2,5501,P010,LN01,SL01,1,407611,330MLCAN 8P COKE CLASSIC X3 MU 11H,X,50077097,0.941,H,10000.0,PAC,9999-12-31,2020-08-21,2020-08-21,10626.99256111,2020-10-07 14:03:19
3,5501,P010,LN01,SL01,1,407733,330MLCAN 8P DT COKE X3 MU 11H,X,50062818,0.941,H,10000.0,PAC,2019-12-23,2019-02-19,2019-12-23,10626.99256111,2020-10-07 14:03:19
4,5501,P010,LN01,SL01,1,465044,TRIAL PRODUCT 120 CS CHEP,X,50062991,2.016,H,10000.0,CS,9999-12-31,2019-03-01,2019-11-14,4960.31746032,2020-10-07 14:03:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9365,5567,P999,LN01,REPK,1,267378,1LNRP X384 SW VP SWT/SWSLT ALDI UDP,X,50076244,1.000,H,10000.0,PAL,9999-12-31,2020-06-15,2020-09-09,10000.00000000,2020-10-07 14:03:19
9366,5567,P999,LN01,REPK,1,271170,330MLCAN 8P SPNS VP SPNS50 SP50 X3 UDP,X,50076245,1.000,H,10000.0,PAC,9999-12-31,2020-06-15,2020-09-09,10000.00000000,2020-10-07 14:03:19
9367,5567,P999,LN01,REPK,1,272764,2.25LEPET X210 DT COKE VP DTC59 CZ41 UDP,X,50076246,1.000,H,10000.0,PAL,9999-12-31,2020-06-15,2020-09-09,10000.00000000,2020-10-07 14:03:19
9368,5567,P999,LN01,REPK,1,277002,150MLCAN 24P SW TONW50 SLTNIC50 ASDA UDP,X,50076473,1.000,H,10000.0,PAC,9999-12-31,2020-07-03,2020-07-03,10000.00000000,2020-10-07 14:03:19


In [70]:
material_runrate.dtypes

Plant                                        object
ProductionVersion                            object
Line                                         object
RecievingSLOC                                object
ProductionVersionStatus                      object
Material                                     object
MaterialDescription                          object
ProcurementType                              object
Group                                        object
ProductionTime                              float64
Unitofmeasureforthestandardvalue             object
BaseQuantity                                float64
UnitofMeasureforActivityOperation            object
ValidTo                                      object
ValidFrom                                    object
LastCheckDate_Out                            object
BaseUnitsPerHour                            float64
DBLoadDate                           datetime64[ns]
dtype: object

In [73]:
material_runrate.loc[(material_runrate.Material.isin(['106513', '245692', '201051']))]

,Plant,ProductionVersion,Line,RecievingSLOC,ProductionVersionStatus,Material,MaterialDescription,ProcurementType,Group,ProductionTime,Unitofmeasureforthestandardvalue,BaseQuantity,UnitofMeasureforActivityOperation,ValidTo,ValidFrom,LastCheckDate_Out,BaseUnitsPerHour,DBLoadDate
1572,5501,P081,LN08,SL01,1,245692,200MLPCH 8X4P CAPSUN ORANGE,X,50062000,17.804,H,10000.0,CS,9999-12-31,2019-01-22,2019-12-12,561.67153449,2020-10-07 14:03:19
1655,5501,P091,LN09,SL01,1,245692,200MLPCH 8X4P CAPSUN ORANGE,X,50066394,17.804,H,10000.0,CS,9999-12-31,2019-07-26,2020-02-12,561.67153449,2020-10-07 14:03:19
1691,5501,P101,LN10,SL01,1,245692,200MLPCH 8X4P CAPSUN ORANGE,X,50066412,17.804,H,10000.0,CS,9999-12-31,2019-07-26,2020-03-05,561.67153449,2020-10-07 14:03:19
5589,5503,P041,LN04,SL01,1,245692,200MLPCH 8X4P CAPSUN ORANGE,X,50061962,15.441,H,10000.0,CS,9999-12-31,2019-01-18,2020-09-23,647.62644906,2020-10-07 14:03:19
5629,5503,P051,LN05,SL01,1,245692,200MLPCH 8X4P CAPSUN ORANGE,X,50061963,15.441,H,10000.0,CS,9999-12-31,2019-01-18,2020-09-23,647.62644906,2020-10-07 14:03:19
6536,5506,P061,LN06,SL01,1,201051,250MLNRG 6X4P DT COKE,X,50051246,10.904,H,10000.0,CS,9999-12-31,2018-01-17,2018-01-17,917.09464417,2020-10-07 14:03:19
6997,5520,P998,LNST,ESTO,1,201051,250MLNRG 6X4P DT COKE,X,50070558,1.000,H,10000.0,CS,2020-09-11,2019-10-17,2020-10-01,10000.00000000,2020-10-07 14:03:19
7518,5520,P999,LNBL,BLIS,1,201051,250MLNRG 6X4P DT COKE,X,50040752,1.000,H,10000.0,CS,2020-09-11,2017-06-05,2020-10-01,10000.00000000,2020-10-07 14:03:19


In [ ]:
material_runrate.loc[(material_runrate.Plant == '5501' )\
                            & (material_runrate.Material == str(material)) \
                            & (material_runrate.Line == line)
                            ]

In [ ]:
line_loading = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'sP_portSummarisedIHLineLoading'))

In [74]:
line_loading.head()

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,090865,5501,LN03,2020_06,Planned,0.0,ZRW
1,Aseptic,090865,5501,LN03,2020_06,Planned,24894.0,ZUC
2,Aseptic,090865,5501,LN03,2020_07,Planned,0.0,ZRW
3,Aseptic,090865,5501,LN03,2020_07,Planned,9842.0,ZUC
4,Aseptic,090865,5501,LN03,2020_08,Planned,0.0,ZRW


In [76]:
line_loading.loc[(line_loading.Site_SAP == 5501)\
                            & (line_loading.Line == 'LN01')
                 
                            ]

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
976,Cans,208269,5501,LN01,2021_03,Planned,6121.89964193,ZRW
977,Cans,208269,5501,LN01,2021_03,Planned,8539.00000000,ZUC
978,Cans,208269,5501,LN01,2021_04,Planned,3064.17602408,ZRW
979,Cans,208269,5501,LN01,2021_04,Planned,4274.00000000,ZUC
1000,Cans,208287,5501,LN01,2020_06,Planned,84473.46846354,ZRW
...,...,...,...,...,...,...,...,...
18368,Cans,,5501,LN01,2020_12,SpareHours,18.45000000,Hours
18369,Cans,,5501,LN01,2021_01,SpareHours,14.35500000,Hours
18370,Cans,,5501,LN01,2021_02,SpareHours,10.86750000,Hours
18371,Cans,,5501,LN01,2021_03,SpareHours,0.00000000,Hours


In [128]:
DEMAND_SHORTFALL_TRESHOLD = 10
INVENTORY_SHORTFALL_THRESHOLD = 5


dP_siteSummarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'dP_siteSummarised')).round(2)

# Demand Shortfalls
demand_plan = dP_siteSummarised.loc[dP_siteSummarised.Uom == 'ZUC'].pivot(index=['Level2Region','Site_SAP','Year_Period','Summary_Portfolio', 'Material', 'Uom'], \
                                      columns='KeyField')['Value'].reset_index()

demand_plan['demand_short_fall'] = -1*(demand_plan['Available'] - demand_plan['Total'])
demand_plan['demand_short_fall_pct'] = round((demand_plan['Available'] - demand_plan['Total'])*-100/ demand_plan['Available'])

demand_shortfalls = demand_plan.loc[demand_plan.demand_short_fall_pct > DEMAND_SHORTFALL_TRESHOLD] 

demand_shortfalls = demand_shortfalls[['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period', 'demand_short_fall']]

demand_shortfalls.Material = demand_shortfalls.Material.astype(str)
demand_shortfalls.Site_SAP = demand_shortfalls.Site_SAP.astype(str)
demand_shortfalls.columns = ['Demand_Site', 'Portfolio', 'Material', 'Year_Period', 'Additional_Units_Required']


# Inventory Shortfalls
pivot_cols = ['KeyField']
pivot_rows = ['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom': 'ZUC'}

iP_Summarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'iP_Summarised')).round(2)

df = iP_Summarised[iP_Summarised.KeyField.isin(['ZUC_ClosingStockTarget', 'ZUC_StockVsTarget'])]

inventory_shortfalls = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)

inventory_shortfalls = pd.DataFrame(inventory_shortfalls.to_records())

inventory_shortfalls.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_shortfalls.columns]

inventory_shortfalls['inventory_shortfall_pct'] = round(inventory_shortfalls['ZUC_StockVsTarget']*-100 / inventory_shortfalls['ZUC_ClosingStockTarget'], 2)

inventory_shortfalls['inventory_shortfall'] = -1*inventory_shortfalls['ZUC_StockVsTarget']

inventory_shortfalls = inventory_shortfalls.loc[inventory_shortfalls.inventory_shortfall_pct > INVENTORY_SHORTFALL_THRESHOLD]

inventory_shortfalls = inventory_shortfalls[['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period', 'inventory_shortfall']]

inventory_shortfalls.Material = inventory_shortfalls.Material.astype(str)
inventory_shortfalls.Site_SAP = inventory_shortfalls.Site_SAP.astype(str)
inventory_shortfalls.columns = ['Inventory_Site', 'Portfolio', 'Material', 'Year_Period', 'Inventory_Units_Required']

In [94]:
shift_lever = pd.read_csv('../input/levers/shift_lever.csv')
shift_lever['Site_SAP'] = shift_lever['Site_SAP'].fillna(0).astype(int).astype(str)
#shift_lever['Material'] = shift_lever['Material'].astype(str)

shift_lever

,Site_SAP,Line,Year_Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,LossFactor,Hours_DownTime,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Additional_Hours
0,5501,LN03,2020_06,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0
1,5501,LN03,2020_07,4,8,3,7,0,672,0,0.25,34,159.50,0.97,464.1450,0
2,5501,LN03,2020_08,4,24,1,5,0,480,192,0.25,34,111.50,0.97,324.4650,0
3,5501,LN03,2020_09,5,24,1,7,0,840,0,0.25,363,119.25,0.97,347.0175,0
4,5501,LN03,2020_10,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,5506,LN02,2020_12,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0
267,5506,LN02,2021_01,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0
268,5506,LN02,2021_02,5,24,1,7,0,840,0,0.25,363,119.25,0.97,347.0175,0
269,5506,LN02,2021_03,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0


In [93]:
shift_lever.loc[shift_lever.Additional_Units_Required > 0]

AttributeError: 'DataFrame' object has no attribute 'Additional_Units_Required'

In [83]:
demand_shortfalls

,Demand_Site,Portfolio,Material,Period,Additional_Units_Required
12,5502,REPACK,106513,2020_06,15197.0
211,5503,REPACK,245692,2020_06,1787.0
603,5506,REPACK,201051,2020_06,12810.0


In [ ]:
group_cols = shift_lever.columns.tolist()
shift_lever = pd.merge(shift_lever, inventory_shortfalls, on=['Portfolio', 'Period', 'Material'], how='left')

In [98]:
ll = line_loading[['Site_SAP', 'Line', 'Material', 'Year_Period']].drop_duplicates()
ll.Site_SAP = ll.Site_SAP.astype(str)

In [100]:
sl = pd.merge(shift_lever, ll, on=['Site_SAP', 'Line','Year_Period'])

In [102]:
sl.loc[(sl.Site_SAP == '5503') & (sl.Material == '245692') ]

,Site_SAP,Line,Year_Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,LossFactor,Hours_DownTime,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Additional_Hours,Material
6100,5503,LN04,2020_06,5,24,1,7,0,840,0,0.25,363,119.25,0.97,347.0175,0,245692
6111,5503,LN04,2020_07,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0,245692
6122,5503,LN04,2020_08,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,245692
6133,5503,LN04,2020_09,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,245692
6144,5503,LN04,2020_10,5,24,1,7,0,840,0,0.25,363,119.25,0.97,347.0175,0,245692
6154,5503,LN04,2020_11,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0,245692
6163,5503,LN04,2020_12,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,245692
6171,5503,LN04,2021_01,4,24,1,7,0,672,0,0.25,34,159.50,0.97,464.1450,0,245692
6178,5503,LN04,2021_02,5,24,1,7,0,840,0,0.25,363,119.25,0.97,347.0175,0,245692
6185,5503,LN04,2021_03,5,24,1,7,0,840,0,0.25,51,197.25,0.97,573.9975,0,245692


In [105]:
input_pickle_dir = '../input/pickle'
pickle_file_name = 'mP_Summarised'
x = pd.read_pickle(os.path.join(input_pickle_dir, pickle_file_name))  
x

,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,Material,KeyField,Value,Uom,ToRepack_Flag
0,,GB,,,North,DIS_NL,5502,2020_06,Monster,207743,GBP_MovementCost,0.00000000,GBP,False
1,,GB,,,North,DIS_NL,5502,2020_06,Monster,207743,PAL,23.84218098,PAL,False
2,,GB,,,North,DIS_NL,5502,2020_06,Monster,207743,Trucks,0.91700696,Trucks,False
3,,GB,,,North,DIS_NL,5502,2020_06,Monster,207743,ZRW,1907.37438822,ZRW,False
4,,GB,,,North,DIS_NL,5502,2020_06,Monster,207743,ZUC,4031.00000000,ZUC,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102935,NL,GB,,,South,15N,5556,2021_03,Aseptic,214634,GBP_MovementCost,0.00000000,GBP,False
102936,NL,GB,,,South,15N,5556,2021_03,Aseptic,214634,PAL,1.21110651,PAL,False
102937,NL,GB,,,South,15N,5556,2021_03,Aseptic,214634,Trucks,0.04658102,Trucks,False
102938,NL,GB,,,South,15N,5556,2021_03,Aseptic,214634,ZRW,203.46588612,ZRW,False


In [106]:
def summarise_df_for_visualisation(df):
    unwanted_cols = ['Material', 'Value']
    cols = [ele for ele in df.columns.tolist() if ele not in unwanted_cols] 

    df = df[cols+['Value']].groupby(cols).sum().round().reset_index()

    return df

In [108]:
df = summarise_df_for_visualisation(x)
df

,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,KeyField,Uom,ToRepack_Flag,Value
0,,GB,,,North,DIS_NL,5502,2020_06,Monster,GBP_MovementCost,GBP,False,0.0
1,,GB,,,North,DIS_NL,5502,2020_06,Monster,PAL,PAL,False,3801.0
2,,GB,,,North,DIS_NL,5502,2020_06,Monster,Trucks,Trucks,False,146.0
3,,GB,,,North,DIS_NL,5502,2020_06,Monster,ZRW,ZRW,False,671962.0
4,,GB,,,North,DIS_NL,5502,2020_06,Monster,ZUC,ZUC,False,699935.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16915,NL,GB,,,South,15N,5556,2021_03,Aseptic,GBP_MovementCost,GBP,False,0.0
16916,NL,GB,,,South,15N,5556,2021_03,Aseptic,PAL,PAL,False,2.0
16917,NL,GB,,,South,15N,5556,2021_03,Aseptic,Trucks,Trucks,False,0.0
16918,NL,GB,,,South,15N,5556,2021_03,Aseptic,ZRW,ZRW,False,269.0


In [109]:
df['FromCountry']

0          
1          
2          
3          
4          
         ..
16915    NL
16916    NL
16917    NL
16918    NL
16919    NL
Name: FromCountry, Length: 16920, dtype: object

In [110]:
conditions = [
        (df['FromCountry'] == 'GB') & (df['ToCountry'] != 'GB'),
        (df['FromCountry'] != 'GB') & (df['ToCountry'] == 'GB'),
        (df['FromCountry'] == 'GB') & (df['ToCountry'] == 'GB') & (df['ToRepack_Flag'] == True),
        (df['FromCountry'] == 'GB') & (df['ToCountry'] == 'GB') & (df['ToRepack_Flag'] == False)
        ]

In [111]:
choices = ['Export', 'Import', 'Domestic Repack', 'IBT']
df['Move_Type'] = np.select(conditions, choices, default='Move_Other')
df

,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,KeyField,Uom,ToRepack_Flag,Value,Move_Type
0,,GB,,,North,DIS_NL,5502,2020_06,Monster,GBP_MovementCost,GBP,False,0.0,Import
1,,GB,,,North,DIS_NL,5502,2020_06,Monster,PAL,PAL,False,3801.0,Import
2,,GB,,,North,DIS_NL,5502,2020_06,Monster,Trucks,Trucks,False,146.0,Import
3,,GB,,,North,DIS_NL,5502,2020_06,Monster,ZRW,ZRW,False,671962.0,Import
4,,GB,,,North,DIS_NL,5502,2020_06,Monster,ZUC,ZUC,False,699935.0,Import
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16915,NL,GB,,,South,15N,5556,2021_03,Aseptic,GBP_MovementCost,GBP,False,0.0,Import
16916,NL,GB,,,South,15N,5556,2021_03,Aseptic,PAL,PAL,False,2.0,Import
16917,NL,GB,,,South,15N,5556,2021_03,Aseptic,Trucks,Trucks,False,0.0,Import
16918,NL,GB,,,South,15N,5556,2021_03,Aseptic,ZRW,ZRW,False,269.0,Import


In [132]:
shift_lever = pd.read_csv('../input/levers/shift_lever.csv')
shift_lever.dtypes

Site_SAP                  int64
Line                     object
Year_Period              object
Weeks_WeeksInPeriod       int64
Shift_Length              int64
Shifts_In_Day             int64
Shift_Days_In_Week        int64
Hours_Overtime            int64
Hours_Crew              float64
Hours_UnCrewed          float64
LossFactor              float64
Hours_DownTime            int64
Hours_SchedPDT          float64
SafetyHours             float64
Hours_TotalAvailable    float64
Additional_Hours        float64
dtype: object

In [145]:
shift_lever = pd.read_csv('../input/levers/shift_lever.csv')
shift_lever['Site_SAP'] = shift_lever['Site_SAP'].fillna(0).astype(int).astype(str)

line_loading = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'sP_portSummarisedIHLineLoading'))
ll = line_loading[['Site_SAP', 'Line', 'Material', 'Year_Period']].drop_duplicates()
ll.Site_SAP = ll.Site_SAP.astype(str)

shift_lever = pd.merge(shift_lever, ll, on=['Site_SAP', 'Line','Year_Period'])

shift_lever

,Site_SAP,Line,Year_Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,LossFactor,Hours_DownTime,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Additional_Hours,Material
0,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,090865
1,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,100691
2,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,101310
3,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,101311
4,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,101312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7511,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,0.25,34,159.50,0.97,464.1450,0.0,444041
7512,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,0.25,34,159.50,0.97,464.1450,0.0,459225
7513,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,0.25,34,159.50,0.97,464.1450,0.0,459228
7514,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,0.25,34,159.50,0.97,464.1450,0.0,462610


In [147]:
shift_lever = pd.merge(shift_lever, demand_shortfalls.drop('Portfolio', axis=1), 
                       left_on=['Year_Period', 'Material'], 
                       right_on=['Year_Period', 'Material'], 
                       how='left')
shift_lever

,Site_SAP,Line,Year_Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,LossFactor,Hours_DownTime,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Additional_Hours,Material,Demand_Site,Additional_Units_Required
0,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,090865,NaN,NaN
1,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,100691,NaN,NaN
2,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,101310,NaN,NaN
3,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,101311,NaN,NaN
4,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,0.0,101312,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7511,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,0.25,34,159.50,0.97,464.1450,0.0,444041,NaN,NaN
7512,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,0.25,34,159.50,0.97,464.1450,0.0,459225,NaN,NaN
7513,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,0.25,34,159.50,0.97,464.1450,0.0,459228,NaN,NaN
7514,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,0.25,34,159.50,0.97,464.1450,0.0,462610,NaN,NaN


In [148]:
shift_lever = pd.merge(shift_lever, 
                       inventory_shortfalls.drop('Portfolio', axis=1), 
                       on=['Year_Period', 'Material'], how='left')

In [149]:
shift_lever

,Site_SAP,Line,Year_Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,...,Hours_DownTime,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Additional_Hours,Material,Demand_Site,Additional_Units_Required,Inventory_Site,Inventory_Units_Required
0,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,090865,NaN,NaN,NaN,NaN
1,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,100691,NaN,NaN,NaN,NaN
2,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,101310,NaN,NaN,NaN,NaN
3,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,101311,NaN,NaN,NaN,NaN
4,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,101312,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7514,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,...,34,159.50,0.97,464.1450,0.0,444041,NaN,NaN,NaN,NaN
7515,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,...,34,159.50,0.97,464.1450,0.0,459225,NaN,NaN,NaN,NaN
7516,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,...,34,159.50,0.97,464.1450,0.0,459228,NaN,NaN,NaN,NaN
7517,5506,LN02,2021_04,4,24,1,7,0,672.0,0.0,...,34,159.50,0.97,464.1450,0.0,462610,NaN,NaN,NaN,NaN


In [150]:
shift_lever.loc[shift_lever.Inventory_Units_Required > 0]

,Site_SAP,Line,Year_Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,...,Hours_DownTime,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Additional_Hours,Material,Demand_Site,Additional_Units_Required,Inventory_Site,Inventory_Units_Required
35,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,414829,NaN,NaN,5503,3696.0
36,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,414829,NaN,NaN,5506,57909.0
38,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,415989,NaN,NaN,5501,3321.0
39,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,416000,NaN,NaN,5501,4317.0
40,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,416000,NaN,NaN,5503,3456.0
41,5501,LN03,2020_06,5,24,1,7,0,840.0,0.0,...,51,197.25,0.97,573.9975,0.0,416000,NaN,NaN,5506,23309.0


In [151]:
iP_Summarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'iP_Summarised'))
iP_Summarised.dtypes

Table                 object
Level2Region          object
Site_SAP              object
SNPPL01               object
KeyField              object
Value                float64
Uom                   object
SiteCountry           object
Summary_Portfolio     object
Material               int64
Line_Portfolio        object
Year_Period           object
dtype: object

In [152]:
iP_Summarised.Material.astype(str)

0         435648
1         435648
2         435648
3         435648
4         435648
           ...  
436251    230116
436252    230115
436253    230116
436254    230116
436255    230115
Name: Material, Length: 436256, dtype: object

In [2]:
input_pickle_dir = '../input/pickle'
ll_file_name = 'sP_portSummarisedIHLineLoading'
ih_file_name = 'sP_portSummarisedIH'

In [9]:
def summarise_df_for_visualisation(df):
    unwanted_cols = ['Material', 'Value']
    cols = [ele for ele in df.columns.tolist() if ele not in unwanted_cols] 

    df = df[cols+['Value']].groupby(cols).sum().round().reset_index()

    return df


In [11]:
pivot_cols = ['KeyField']
pivot_rows = ['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'Hours'}

df = pd.read_pickle(os.path.join(input_pickle_dir, pickle_file_name))
df = summarise_df_for_visualisation(df)

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)




production_line_plan = pd.DataFrame(wide_df.to_records())

production_line_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                    for hdr in production_line_plan.columns]

#Calcuated fields
production_line_plan['Capacity_Utilisation'] = production_line_plan['Consumed']*100 / production_line_plan['Capacity']

production_line_plan['Available_Vol'] = production_line_plan['EstimatedUnconsumedCapacity']
production_line_plan['Available_Vol_with_Safety_Hours'] = production_line_plan['EstimatedUnconsumedCapacity'] + production_line_plan['EstimatedHoursSpareCapacity']

In [3]:
ll = df = pd.read_pickle(os.path.join(input_pickle_dir, ll_file_name))
ih = df = pd.read_pickle(os.path.join(input_pickle_dir, ih_file_name))

In [4]:
ll.head()

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,090865,5501,LN03,2020_06,Planned,0.0,ZRW
1,Aseptic,090865,5501,LN03,2020_06,Planned,24894.0,ZUC
2,Aseptic,090865,5501,LN03,2020_07,Planned,0.0,ZRW
3,Aseptic,090865,5501,LN03,2020_07,Planned,9842.0,ZUC
4,Aseptic,090865,5501,LN03,2020_08,Planned,0.0,ZRW


In [17]:
ll.head().loc[ll.Value > 10000].Value

1    24894.0
Name: Value, dtype: float64

In [26]:
SERVER_NAME = 'CVAPWPEQR001'
DATABASE_NAME = 'PACSQL_MTP'
SUPPLY_PLAN_INHOUSE_TABLE = 'dbo.temp_sP_portSummarisedIH_materiallevel'
PRODUCTION_CAPACITY_TABLE = 'production.Extract_ecc_ProductionCapability'

import pyodbc

def get_inhouse_production_plan():
    try:      

        conn = pyodbc.connect('Driver={SQL Server};'
                            f'Server={SERVER_NAME};'
                            f'Database={DATABASE_NAME};'
                            'Trusted_Connection=yes;')

        sql = f'''
                SELECT   
                    IH.[Site_SAP]
                    ,IH.[Line]
                    ,IH.[Year_Period]
                    ,IH.[Summary_Portfolio]
                    ,IH.[Material]
                    ,IH.[Uom]
                    ,IH.[Value]
                    ,PC.[BaseUnitsPerHour]
                    ,ROUND(IH.[Value] / PC.[BaseUnitsPerHour], 2) AS ConsumedHours
                FROM [PACSQL_MTP].{SUPPLY_PLAN_INHOUSE_TABLE} IH
                INNER JOIN [PACSQL_MTP].{PRODUCTION_CAPACITY_TABLE} PC
                    ON IH.Site_SAP = PC.Plant
                    AND IH.Line = PC.Line
                    AND IH.Material = PC.Material
                    WHERE 1=1
                    AND GETDATE() BETWEEN PC.ValidFrom AND PC.ValidTo
                    AND IH.Uom = 'ZUC'
            '''      
        
        data = pd.read_sql(sql,conn)           
        
        conn.close()
    except Exception as e:        
        print(e)

    return data

In [27]:
inhouse_production = get_inhouse_production_plan()

In [32]:
material = '203380'
periods = ['2020_06']


manu = inhouse_production[(inhouse_production.Material == material)]

In [90]:
input_pickle_dir

'../input/pickle'

In [93]:
ll = pd.read_pickle(os.path.join(input_pickle_dir, ll_file_name))
ll

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,090865,5501,LN03,2020_06,Planned,0.0,ZRW
1,Aseptic,090865,5501,LN03,2020_06,Planned,24894.0,ZUC
2,Aseptic,090865,5501,LN03,2020_07,Planned,0.0,ZRW
3,Aseptic,090865,5501,LN03,2020_07,Planned,9842.0,ZUC
4,Aseptic,090865,5501,LN03,2020_08,Planned,0.0,ZRW
...,...,...,...,...,...,...,...,...
20783,Water,,5546,LN01,2021_02,SpareHours,0.0,Hours
20784,Water,,5546,LN01,2021_03,SpareHours,0.0,Hours
20785,Water,,5546,LN01,2021_04,SpareHours,0.0,Hours
20786,Water,,5546,LN01,2021_05,SpareHours,0.0,Hours


In [57]:
ll = ll.loc[ll.KeyField.isin(['Capacity', 'Consumed', 'SpareHours'])].drop(['Material','Line_Portfolio','Uom'], axis=1)

In [58]:
pd.pivot_table(ll, values='Value', index=['Site_SAP', 'Line', 'Year_Period'],
                        columns='KeyField', aggfunc=sum).reset_index()

KeyField,Site_SAP,Line,Year_Period,Capacity,Consumed,SpareHours
0,5501,LN01,2020_06,574.007,574.0065799,17.7525
1,5501,LN01,2020_07,464.156,464.1556164,14.3550
2,5501,LN01,2020_08,464.156,464.1554031,14.3550
3,5501,LN01,2020_09,347.025,315.5904816,10.7325
4,5501,LN01,2020_10,464.852,464.8517696,14.3550
...,...,...,...,...,...,...
351,5546,LN01,2021_02,335.377,202.0942381,0.0000
352,5546,LN01,2021_03,839.000,278.8303948,0.0000
353,5546,LN01,2021_04,672.000,53.9290428,0.0000
354,5546,LN01,2021_05,672.000,12.2938037,0.0000


In [63]:
x = pd.pivot_table(ll, values='Value', index=['Site_SAP', 'Line', 'Year_Period'],
                        columns='KeyField', aggfunc=sum).reset_index()

In [68]:
SUPPLY_PLAN_INHOUSE_TABLE = 'dbo.temp_sP_portSummarisedIH_materiallevel'
SUPPLY_PLAN_INHOUSE_LINELOADING_TABLE = 'dbo.temp_sP_portSummarisedIHLineLoading_materiallevel'

In [74]:
sP_portSummarisedIH = get_datatable_from_db(table_name=SUPPLY_PLAN_INHOUSE_TABLE)    
sP_portSummarisedIH.to_pickle(os.path.join(input_pickle_dir, 'sP_portSummarisedIH'))

sP_portSummarisedIHLineLoading = get_datatable_from_db(table_name=SUPPLY_PLAN_INHOUSE_LINELOADING_TABLE)
sP_portSummarisedIHLineLoading.to_pickle(os.path.join(input_pickle_dir, 'sP_portSummarisedIHLineLoading'))

Fetching data for dbo.temp_sP_portSummarisedIH_materiallevel
Fetching data for dbo.temp_sP_portSummarisedIHLineLoading_materiallevel


In [92]:
sP_portSummarisedIHLineLoading = get_datatable_from_db(table_name=SUPPLY_PLAN_INHOUSE_LINELOADING_TABLE)
#sP_portSummarisedIHLineLoading
sP_portSummarisedIHLineLoading.to_pickle(os.path.join(input_pickle_dir, 'sP_portSummarisedIHLineLoading'))

Fetching data for dbo.temp_sP_portSummarisedIHLineLoading_materiallevel


In [81]:
a = ["s", "f", "r"]
b = ["1", "3", "3"]

print(f''' {[f"{a}-{b}" for a, b in zip(a, b)]} wq  sdasd sdsad ''')

 ['s-1', 'f-3', 'r-3'] wq  sdasd sdsad 


In [97]:
' '.join(a)

's f r'

In [98]:
import dash_html_components as html

In [101]:
a.extend(['xxx', html.Br()])

In [18]:
def get_next_n_periods(this_period, n) :
    this_month = int(this_period.split('_')[1])
    this_year = int(this_period.split('_')[0])
    
    period_list = [this_period]
    
    for i in range(n):    
        next_year = this_year    
        if this_month==12:
            next_year = this_year+1
            next_month = 1
        else:
            next_month = this_month+1
        
        period_list.append(f"{next_year}_{next_month:02d}")
        
        this_year=next_year
        this_month=next_month
    
    return period_list


In [25]:
def get_previous_n_periods(this_period, n) :
    this_month = int(this_period.split('_')[1])
    this_year = int(this_period.split('_')[0])
    
    period_list = [this_period]
    
    for i in range(n):    
        prev_year = this_year    
        if this_month==1:
            prev_year = this_year-1
            prev_month = 12
        else:
            prev_month = this_month-1
        
        period_list.append(f"{prev_year}_{prev_month:02d}")
        
        this_year=prev_year
        this_month=prev_month
    
    period_list.reverse()
    
    return period_list


In [29]:
get_previous_n_periods('2020_04', 5)

['2019_11', '2019_12', '2020_01', '2020_02', '2020_03', '2020_04']

In [32]:
ll.drop_duplicates(subset=['Line_Portfolio'])

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,090865,5501,LN03,2020_06,Planned,0.00,ZRW
872,Cans,106513,5502,LN02,2020_06,Planned,0.00,ZRW
5768,Capri Sun,243336,5501,LN08,2020_06,Planned,367.66,ZRW
6650,GB,269269,5501,LN07,2020_06,Planned,23930.30,ZRW
6800,Glass,201050,5506,LN06,2020_11,Planned,20755.41,ZRW
8244,LPET,204231,5502,LN03,2020_06,Planned,186621.75,ZRW
12546,NL,262165,5501,LN06,2020_06,Planned,27817.10,ZRW
12552,Oasis,223720,5506,LN05,2020_06,Planned,3079.43,ZRW
13008,Postmix,206812,5506,LN03,2020_06,Planned,3211.91,ZRW
13402,REPACK,470300,5546,LN01,2020_06,Planned,3779.89,ZRW


In [36]:
x = np.nan

In [37]:
x

nan

In [1]:
import pyodbc
import pandas as pd

PROJECTED_DEMAND_TABLE = 'model.MTP_WF003_02_ProjectedDemand'
INVENTORY_TABLE = 'dbo.temp_iP_Summarised_materiallevel_REFRESH'
SERVER_NAME = 'CVAPWPEQR001'
DATABASE_NAME = 'PACSQL_MTP'

In [235]:
INVENTORY_TABLE

'dbo.temp_iP_Summarised_materiallevel_REFRESH'

In [257]:
PROJECTED_DEMAND_TABLE = 'model.MTP_WF003_02_ProjectedDemand_REFRESH'

In [258]:
def get_DSI_table():
    
    data = None
    
    try:      

        conn = pyodbc.connect('Driver={SQL Server};'
                            f'Server={SERVER_NAME};'
                            f'Database={DATABASE_NAME};'
                            'Trusted_Connection=yes;')

        sql = f'''               

                SELECT	  
                    CASE PD.[Site_SAP]
                            WHEN 'P67' THEN '5567'
                            WHEN 'P69' THEN '5569'
                            WHEN 'P14' THEN '5504'
                            ELSE PD.[Site_SAP]
                        END AS Site_SAP
                    ,CASE PD.[Site_SAP]
                            WHEN 'P67' THEN 'North'
                            WHEN 'P69' THEN 'North'
                            WHEN 'P14' THEN 'South'
                            ELSE PD.[Level2Region]
                        END AS Level2Region
                    ,PD.[Summary_Portfolio] 
                    ,PD.[Material]
                    ,CONCAT([Year],'_',format (CAST(PD.[CCEP_Period] AS INT), '0#')) AS Year_Period
                    ,CASE PD.[CCEP_Period] % 3 WHEN 0 THEN 5 ELSE 4 END AS BucketsInPeriod
                    ,PD.[Value] AS ZUC_TotalDemand
                    ,ROUND(PD.[Value] / ((CASE PD.[CCEP_Period] % 3 WHEN 0 THEN 5 ELSE 4 END)*7),2) AS ZUC_PeriodDailyDemand
                    ,IP.[Value] AS ZUC_ClosingStock
                    ,IP_target.[Value] AS ZUC_ClosingStock_Target
                    --,ROUND(IP.[Value] / ROUND(PD.[Value] / ((CASE PD.[CCEP_Period] % 3 WHEN 0 THEN 5 ELSE 4 END)*7),2),2) AS Days_DSI
                    --,ROUND(IP_target.[Value] / ROUND(PD.[Value] / ((CASE PD.[CCEP_Period] % 3 WHEN 0 THEN 5 ELSE 4 END)*7),2),2) AS Days_DSI_Target
                    
                FROM [PACSQL_MTP].{PROJECTED_DEMAND_TABLE} PD
                INNER JOIN [PACSQL_MTP].{INVENTORY_TABLE} IP
                    ON IP.Year_Period = CONCAT(PD.[Year],'_',format (CAST(PD.[CCEP_Period] AS INT), '0#'))
                    AND IP.Material = PD.[Material]
                    AND IP.Site_SAP = PD.Site_SAP
                INNER JOIN [PACSQL_MTP].{INVENTORY_TABLE} IP_target
                    ON IP_target.Year_Period = CONCAT(PD.[Year],'_',format (CAST(PD.[CCEP_Period] AS INT), '0#'))
                    AND IP_target.Material = PD.[Material]
                    AND IP_target.Site_SAP = PD.Site_SAP
                WHERE 1=1
                    AND PD.[Table] = 'Demand_Site_Material'
                    AND PD.[KeyField] = 'ZUC_TotalDemand'
                    AND IP.[KeyField] = 'ZUC_ClosingStock'
                    AND IP_target.[KeyField] = 'ZUC_ClosingStockTarget'  
                    AND PD.[Summary_Portfolio] = 'LPET'
                    AND PD.[CCEP_Period] = '12'
                    AND PD.[Year]='2020'                    

            '''      
        
        data = pd.read_sql(sql,conn)                
        
        conn.close()
    except Exception as e:
        print(e)

        
    return data

In [259]:
dsi_table = get_DSI_table()
dsi_table = dsi_table.fillna(0)

In [260]:
dsi_table

,Site_SAP,Level2Region,Summary_Portfolio,Material,Year_Period,BucketsInPeriod,ZUC_TotalDemand,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target
0,5501,South,LPET,232676,2020_12,5,902,25,0,0
1,5501,South,LPET,219079,2020_12,5,1495,42,105,105
2,5501,South,LPET,217042,2020_12,5,929,26,299,299
3,5501,South,LPET,459225,2020_12,5,3155,90,1339,1339
4,5556,South,LPET,219944,2020_12,5,711,20,91,91
...,...,...,...,...,...,...,...,...,...,...
410,5567,North,LPET,219945,2020_12,5,48451,1384,4142,4142
411,5567,North,LPET,408022,2020_12,5,34452,984,4271,4271
412,5503,Scotland,LPET,262627,2020_12,5,10880,310,4680,4680
413,5503,Scotland,LPET,219943,2020_12,5,18127,517,5813,5813


In [261]:
dsi_table.ZUC_ClosingStock.astype(int).sum()

2453650

In [242]:
#dsi_table = dsi_table[dsi_table.Material.isin(['215671'])]
dsi_table.loc[dsi_table.Year_Period == '2020_12'].ZUC_ClosingStock.astype(float).sum()

46656.0

In [166]:
dsi_table = dsi_table[['Summary_Portfolio', 'Level2Region', 'Year_Period', 'ZUC_PeriodDailyDemand',	'ZUC_ClosingStock','ZUC_ClosingStock_Target']]
dsi_table['ZUC_ClosingStock'] = dsi_table['ZUC_ClosingStock'].astype(float)
dsi_table['ZUC_ClosingStock_Target'] = dsi_table['ZUC_ClosingStock_Target'].astype(float)

dsi_table

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target
0,LPET,South,2020_12,42.94,1026.0,1026.0
1,LPET,South,2020_12,148.06,160.0,160.0
2,LPET,South,2020_12,51.57,299.0,299.0
3,LPET,South,2020_12,144.86,1226.0,1226.0
4,LPET,South,2020_12,29.97,91.0,91.0
...,...,...,...,...,...,...
960,LPET,South,2021_03,19.77,59.0,59.0
961,LPET,North,2021_01,163.86,13173.0,13173.0
962,LPET,North,2021_03,364.89,21576.0,21576.0
963,LPET,North,2021_03,401.26,29371.0,29371.0


In [183]:
dsi_group = dsi_table.groupby(['Summary_Portfolio', 'Level2Region', 'Year_Period']).sum().reset_index()
dsi_group

,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target
0,LPET,North,2020_12,117839.60,529519.0,529519.0
1,LPET,North,2021_01,72495.33,709375.0,709375.0
2,LPET,North,2021_02,94054.52,846791.0,843423.0
3,LPET,North,2021_03,97138.07,825394.0,825394.0
4,LPET,North,2021_04,100452.99,754500.0,747255.0
5,LPET,Scotland,2020_12,20018.57,145425.0,145425.0
6,LPET,Scotland,2021_01,12640.44,201602.0,201602.0
7,LPET,Scotland,2021_02,16208.17,244053.0,243734.0
8,LPET,Scotland,2021_03,16824.14,234804.0,234805.0
9,LPET,Scotland,2021_04,18041.11,215338.0,215339.0


In [184]:
dsi_group.sort_values('Year_Period')

,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target
0,LPET,North,2020_12,117839.60,529519.0,529519.0
5,LPET,Scotland,2020_12,20018.57,145425.0,145425.0
10,LPET,South,2020_12,120961.93,673204.0,673204.0
1,LPET,North,2021_01,72495.33,709375.0,709375.0
6,LPET,Scotland,2021_01,12640.44,201602.0,201602.0
11,LPET,South,2021_01,71923.56,874523.0,874523.0
2,LPET,North,2021_02,94054.52,846791.0,843423.0
7,LPET,Scotland,2021_02,16208.17,244053.0,243734.0
12,LPET,South,2021_02,92161.52,1162302.0,1029356.0
3,LPET,North,2021_03,97138.07,825394.0,825394.0


In [168]:
dsi_north = dsi_group[dsi_group.Level2Region == 'North']
dsi_south = dsi_group[dsi_group.Level2Region == 'South']
dsi_scotland = dsi_group[dsi_group.Level2Region == 'Scotland']

In [156]:
dsi_north['NEXT_PeriodDailyDemand'] = dsi_north['ZUC_PeriodDailyDemand'].shift(-1)
dsi_north

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,NEXT_PeriodDailyDemand
0,LPET,North,2020_12,3494.94,15803.0,15803.0,2542.21
1,LPET,North,2021_01,2542.21,15738.0,15738.0,3188.82
2,LPET,North,2021_02,3188.82,16119.0,16119.0,3188.86
3,LPET,North,2021_03,3188.86,16060.0,16060.0,3188.86
4,LPET,North,2021_04,3188.86,15630.0,15630.0,NaN


In [160]:
dsi_south['NEXT_PeriodDailyDemand'] = dsi_south['ZUC_PeriodDailyDemand'].shift(-1)
dsi_south

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,NEXT_PeriodDailyDemand
5,LPET,South,2020_12,1387.71,23328.0,23328.0,1009.43
6,LPET,South,2021_01,1009.43,23233.0,23233.0,1266.14
7,LPET,South,2021_02,1266.14,23795.0,23795.0,1266.14
8,LPET,South,2021_03,1266.14,23707.0,23707.0,1266.14
9,LPET,South,2021_04,1266.14,23074.0,23074.0,NaN


In [161]:
dsi_scotland['NEXT_PeriodDailyDemand'] = dsi_scotland['ZUC_PeriodDailyDemand'].shift(-1)
dsi_scotland

,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,NEXT_PeriodDailyDemand


In [169]:
dsi_north['Days_DSI'] = dsi_north['ZUC_ClosingStock'] / dsi_north['ZUC_PeriodDailyDemand'].shift(-1)
dsi_north['Days_DSI_Target'] = dsi_north['ZUC_ClosingStock_Target'] / dsi_north['ZUC_PeriodDailyDemand'].shift(-1)
dsi_north

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Days_DSI,Days_DSI_Target
0,LPET,North,2020_12,117839.60,529519.0,529519.0,7.304181,7.304181
1,LPET,North,2021_01,72495.33,709375.0,709375.0,7.542168,7.542168
2,LPET,North,2021_02,94054.52,846791.0,843423.0,8.717396,8.682723
3,LPET,North,2021_03,97138.07,825394.0,825394.0,8.216719,8.216719
4,LPET,North,2021_04,100452.99,754500.0,747255.0,NaN,NaN


In [170]:
dsi_south['Days_DSI'] = dsi_south['ZUC_ClosingStock'] / dsi_south['ZUC_PeriodDailyDemand'].shift(-1)
dsi_south['Days_DSI_Target'] = dsi_south['ZUC_ClosingStock_Target'] / dsi_south['ZUC_PeriodDailyDemand'].shift(-1)
dsi_south

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Days_DSI,Days_DSI_Target
10,LPET,South,2020_12,120961.93,673204.0,673204.0,9.359993,9.359993
11,LPET,South,2021_01,71923.56,874523.0,874523.0,9.489025,9.489025
12,LPET,South,2021_02,92161.52,1162302.0,1029356.0,12.142583,10.753694
13,LPET,South,2021_03,95721.15,1042413.0,1042412.0,10.660888,10.660878
14,LPET,South,2021_04,97779.19,940799.0,936974.0,NaN,NaN


In [171]:
dsi_scotland['Days_DSI'] = dsi_scotland['ZUC_ClosingStock'] / dsi_scotland['ZUC_PeriodDailyDemand'].shift(-1)
dsi_scotland['Days_DSI_Target'] = dsi_scotland['ZUC_ClosingStock_Target'] / dsi_scotland['ZUC_PeriodDailyDemand'].shift(-1)
dsi_scotland

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Days_DSI,Days_DSI_Target
5,LPET,Scotland,2020_12,20018.57,145425.0,145425.0,11.504742,11.504742
6,LPET,Scotland,2021_01,12640.44,201602.0,201602.0,12.438295,12.438295
7,LPET,Scotland,2021_02,16208.17,244053.0,243734.0,14.506120,14.487160
8,LPET,Scotland,2021_03,16824.14,234804.0,234805.0,13.014942,13.014997
9,LPET,Scotland,2021_04,18041.11,215338.0,215339.0,NaN,NaN


In [172]:
dsi_group = pd.concat([dsi_north, dsi_south, dsi_scotland])

In [173]:
dsi_group.sort_values(by=['Year_Period'])

,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Days_DSI,Days_DSI_Target
0,LPET,North,2020_12,117839.60,529519.0,529519.0,7.304181,7.304181
10,LPET,South,2020_12,120961.93,673204.0,673204.0,9.359993,9.359993
5,LPET,Scotland,2020_12,20018.57,145425.0,145425.0,11.504742,11.504742
1,LPET,North,2021_01,72495.33,709375.0,709375.0,7.542168,7.542168
11,LPET,South,2021_01,71923.56,874523.0,874523.0,9.489025,9.489025
6,LPET,Scotland,2021_01,12640.44,201602.0,201602.0,12.438295,12.438295
2,LPET,North,2021_02,94054.52,846791.0,843423.0,8.717396,8.682723
12,LPET,South,2021_02,92161.52,1162302.0,1029356.0,12.142583,10.753694
7,LPET,Scotland,2021_02,16208.17,244053.0,243734.0,14.506120,14.487160
3,LPET,North,2021_03,97138.07,825394.0,825394.0,8.216719,8.216719


In [174]:
dsi_regional = pd.pivot(dsi_group, 
            values=['Days_DSI', 'Days_DSI_Target'],
            index=[ 'Summary_Portfolio', 'Year_Period'],
            columns=['Level2Region']
            )

dsi_regional = pd.DataFrame(dsi_regional.to_records())
dsi_regional.columns = [hdr.replace("('Days_DSI_Target', '", "Target_").replace("('Days_DSI', '", "Planned_").replace("')", "") for hdr in dsi_regional.columns]

In [175]:
dsi_regional

,Summary_Portfolio,Year_Period,Planned_North,Planned_Scotland,Planned_South,Target_North,Target_Scotland,Target_South
0,LPET,2020_12,7.304181,11.504742,9.359993,7.304181,11.504742,9.359993
1,LPET,2021_01,7.542168,12.438295,9.489025,7.542168,12.438295,9.489025
2,LPET,2021_02,8.717396,14.506120,12.142583,8.682723,14.487160,10.753694
3,LPET,2021_03,8.216719,13.014942,10.660888,8.216719,13.014997,10.660878
4,LPET,2021_04,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
dsi_national = dsi_table[['Summary_Portfolio', 'Year_Period', 
            'ZUC_PeriodDailyDemand', 'ZUC_ClosingStock', 
            'ZUC_ClosingStock_Target']].groupby(['Summary_Portfolio', 'Year_Period']).sum().reset_index()

dsi_national['Planned_National'] = dsi_national['ZUC_ClosingStock'] / dsi_national['ZUC_PeriodDailyDemand'].shift(-1)
dsi_national['Target_National'] = dsi_national['ZUC_ClosingStock_Target'] / dsi_national['ZUC_PeriodDailyDemand'].shift(-1)

In [177]:
dsi_national

,Summary_Portfolio,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Planned_National,Target_National
0,LPET,2020_12,258820.10,1348148.0,1348148.0,8.583686,8.583686
1,LPET,2021_01,157059.33,1785500.0,1785500.0,8.820585,8.820585
2,LPET,2021_02,202424.21,2253146.0,2116513.0,10.745469,10.093853
3,LPET,2021_03,209683.36,2102611.0,2102611.0,9.722010,9.722010
4,LPET,2021_04,216273.29,1910637.0,1899568.0,NaN,NaN


In [178]:
dsi_national = dsi_national[['Summary_Portfolio', 'Year_Period', 'Planned_National', 'Target_National']]

dsi_plot = pd.merge(dsi_regional, dsi_national, on=['Summary_Portfolio', 'Year_Period'], how='inner')

In [179]:
dsi_plot

,Summary_Portfolio,Year_Period,Planned_North,Planned_Scotland,Planned_South,Target_North,Target_Scotland,Target_South,Planned_National,Target_National
0,LPET,2020_12,7.304181,11.504742,9.359993,7.304181,11.504742,9.359993,8.583686,8.583686
1,LPET,2021_01,7.542168,12.438295,9.489025,7.542168,12.438295,9.489025,8.820585,8.820585
2,LPET,2021_02,8.717396,14.506120,12.142583,8.682723,14.487160,10.753694,10.745469,10.093853
3,LPET,2021_03,8.216719,13.014942,10.660888,8.216719,13.014997,10.660878,9.722010,9.722010
4,LPET,2021_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
summary_portfolio = 'LPET'

In [113]:
plot_df = dsi_plot.loc[dsi_plot.Summary_Portfolio == summary_portfolio] 

In [114]:
plot_df.head()

,Summary_Portfolio,Year_Period,Planned_North,Planned_Scotland,Planned_South,Target_North,Target_Scotland,Target_South,Planned_National,Target_National
0,LPET,2020_12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LPET,2021_01,6.019835,10.070749,7.229737,6.019835,10.070749,7.229737,6.898614,6.898614
2,LPET,2021_02,11.680628,19.307318,16.160240,11.634170,19.282082,14.311805,14.345827,13.475882
3,LPET,2021_03,8.775697,14.486768,11.310718,8.775697,14.486830,11.310708,10.387152,10.387152
4,LPET,2021_04,7.767295,12.799347,9.828538,7.692710,12.799406,9.788579,9.112011,9.059221


In [118]:
plot_df = plot_df.dropna()

In [180]:
import plotly.graph_objs as go
import matplotlib
import seaborn as sns

fig = go.Figure()
#fig = go.FigureWidget(fig)


# fig.update_layout(
#     template="simple_white",
#     xaxis=dict(title_text="Year Period"),
#     yaxis=dict(title_text="Days", showgrid= True)    
# )

bar_cols = ['Planned_North', 'Planned_South', 'Planned_Scotland', 'Planned_National']
colors = sns.color_palette("Oranges_r").as_hex()

for column, color in zip(bar_cols, colors):       
    fig.add_trace(
        go.Bar(x=plot_df['Year_Period'], y=plot_df[column], name=column, marker_color=color),
    )

fig.add_trace(
        go.Scatter(x=plot_df.Year_Period ,y=plot_df.Target_National, name='Target_National')
) 

fig.show()

In [82]:
dsi_group[['Summary_Portfolio', 'Year_Period', 
           'Days_DSI_C', 'Days_DSI_Target_C']].groupby(['Summary_Portfolio', 'Year_Period']).sum()

Days_DSI_C  Days_DSI_Target_C
Summary_Portfolio Year_Period                                
Cans - Board      2020_12       5.16900199         4.59110153
                  2021_01      17.03632612        16.48889994
                  2021_02      17.08373754        15.28359891
                  2021_03      14.15549257        14.15555624
                  2021_04      15.37112888        14.89749932
Cans - Shrink     2020_12       9.77651625         3.35095707
                  2021_01      29.92201135         8.57862277
                  2021_02      10.83080508         7.11638776
                  2021_03       6.62213270         6.62229847
                  2021_04      10.50830939         6.69789738
GB                2020_12      14.91365777        14.91365777

In [102]:
pd.pivot_table(dsi_table.loc[dsi_table.Summary_Portfolio == 'Aseptic'],
         values=['ZUC_PeriodDailyDemand', 'ZUC_ClosingStock', 'ZUC_ClosingStock_Target'],
         index=['Level2Region'],
         columns=['Year_Period'],
         aggfunc=sum)

ZUC_PeriodDailyDemand                                    
Year_Period                2020_12  2021_01  2021_02  2021_03  2021_04
Level2Region                                                          
North                      2893.29  1053.70  1239.06  1280.10  1334.99
Scotland                     86.03    63.07    79.75    88.63   107.00
South                      1998.57   894.11  1028.75  1110.20  1246.99

In [112]:
dsi_table.ZUC_ClosingStock = dsi_table.ZUC_ClosingStock.astype(float)
dsi_table.ZUC_ClosingStock_Target = dsi_table.ZUC_ClosingStock_Target.astype(float)

In [118]:
dsi_group = dsi_table[['Summary_Portfolio', 'Level2Region', 'Year_Period', 
           'ZUC_PeriodDailyDemand', 'ZUC_ClosingStock', 
           'ZUC_ClosingStock_Target']].groupby(['Summary_Portfolio', 'Level2Region', 'Year_Period']).sum().reset_index()

In [119]:
dsi_group['Days_DSI'] = dsi_group['ZUC_ClosingStock'] / dsi_group['ZUC_PeriodDailyDemand']
dsi_group['Days_DSI_Target'] = dsi_group['ZUC_ClosingStock_Target'] / dsi_group['ZUC_PeriodDailyDemand']
dsi_group

,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Days_DSI,Days_DSI_Target
0,Aseptic,North,2020_12,2893.29,28894.0,21889.0,9.98655510,7.56543589
1,Aseptic,North,2021_01,1053.70,22151.0,21939.0,21.02211256,20.82091677
2,Aseptic,North,2021_02,1239.06,26035.0,26035.0,21.01189611,21.01189611
3,Aseptic,North,2021_03,1280.10,29857.0,29857.0,23.32395907,23.32395907
4,Aseptic,North,2021_04,1334.99,32051.0,30909.0,24.00841954,23.15298242
...,...,...,...,...,...,...,...,...
232,Water,South,2020_12,12450.64,142002.0,142002.0,11.40519684,11.40519684
233,Water,South,2021_01,8675.11,119528.0,119528.0,13.77826909,13.77826909
234,Water,South,2021_02,10280.76,150720.0,150439.0,14.66039476,14.63306215
235,Water,South,2021_03,11822.33,203448.0,195978.0,17.20879048,16.57693534


In [123]:
dsi_group

,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Days_DSI,Days_DSI_Target
0,Aseptic,North,2020_12,2893.29,28894.0,21889.0,9.98655510,7.56543589
1,Aseptic,North,2021_01,1053.70,22151.0,21939.0,21.02211256,20.82091677
2,Aseptic,North,2021_02,1239.06,26035.0,26035.0,21.01189611,21.01189611
3,Aseptic,North,2021_03,1280.10,29857.0,29857.0,23.32395907,23.32395907
4,Aseptic,North,2021_04,1334.99,32051.0,30909.0,24.00841954,23.15298242
...,...,...,...,...,...,...,...,...
232,Water,South,2020_12,12450.64,142002.0,142002.0,11.40519684,11.40519684
233,Water,South,2021_01,8675.11,119528.0,119528.0,13.77826909,13.77826909
234,Water,South,2021_02,10280.76,150720.0,150439.0,14.66039476,14.63306215
235,Water,South,2021_03,11822.33,203448.0,195978.0,17.20879048,16.57693534


In [131]:
dsi_group.loc[dsi_group.Summary_Portfolio == 'Aseptic']

,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Days_DSI,Days_DSI_Target
0,Aseptic,North,2020_12,2893.29,28894.0,21889.0,9.98655510,7.56543589
1,Aseptic,North,2021_01,1053.70,22151.0,21939.0,21.02211256,20.82091677
2,Aseptic,North,2021_02,1239.06,26035.0,26035.0,21.01189611,21.01189611
3,Aseptic,North,2021_03,1280.10,29857.0,29857.0,23.32395907,23.32395907
4,Aseptic,North,2021_04,1334.99,32051.0,30909.0,24.00841954,23.15298242
5,Aseptic,Scotland,2020_12,86.03,2991.0,2991.0,34.76694176,34.76694176
6,Aseptic,Scotland,2021_01,63.07,3735.0,3735.0,59.21991438,59.21991438
7,Aseptic,Scotland,2021_02,79.75,4876.0,4876.0,61.14106583,61.14106583
8,Aseptic,Scotland,2021_03,88.63,5737.0,5737.0,64.72977547,64.72977547
9,Aseptic,Scotland,2021_04,107.00,6388.0,6388.0,59.70093458,59.70093458


In [156]:
plot_df = pd.pivot(dsi_group.loc[dsi_group.Summary_Portfolio == 'Aseptic'], 
                     values=['Days_DSI', 'Days_DSI_Target'],
                     index=['Year_Period'],
                     columns=['Level2Region']
                    )

plot_df

plot_df = pd.DataFrame(plot_df.to_records())

plot_df.columns = [hdr.replace("('Days_DSI_Target', '", "Target_").replace("('Days_DSI', '", "Planned_").replace("')", "") for hdr in plot_df.columns]

plot_df['Palnned_National'] = plot_df['Planned_North'] +  plot_df['Planned_Scotland'] + plot_df['Planned_South']
plot_df['Target_National'] = plot_df['Target_North'] +  plot_df['Target_Scotland'] + plot_df['Target_South']

plot_df = plot_df.fillna(0)

plot_df

,Year_Period,Planned_North,Planned_Scotland,Planned_South,Target_North,Target_Scotland,Target_South,Palnned_National,Target_National
0,2020_12,9.98655510,34.76694176,11.19100157,7.56543589,34.76694176,9.40272295,55.94449843,51.73510060
1,2021_01,21.02211256,59.21991438,23.83151961,20.82091677,59.21991438,22.96697274,104.07354655,103.00780389
2,2021_02,21.01189611,61.14106583,24.66974484,21.01189611,61.14106583,24.55309842,106.82270678,106.70606037
3,2021_03,23.32395907,64.72977547,26.48982165,23.32395907,64.72977547,26.44298325,114.54355619,114.49671778
4,2021_04,24.00841954,59.70093458,34.74767240,23.15298242,59.70093458,25.27927249,118.45702651,108.13318949


In [155]:
plot_df

,Year_Period,North,Scotland,South,National
0,2020_12,9.98655510,34.76694176,11.19100157,55.94449843
1,2021_01,21.02211256,59.21991438,23.83151961,104.07354655
2,2021_02,21.01189611,61.14106583,24.66974484,106.82270678
3,2021_03,23.32395907,64.72977547,26.48982165,114.54355619
4,2021_04,24.00841954,59.70093458,34.74767240,118.45702651


In [32]:
import plotly.graph_objs as go
import seaborn as sns

fig = go.Figure()


bar_cols = ['Planned_North', 'Planned_South', 'Planned_Scotland', 'Palnned_National'] #plot_df.columns.tolist()
#bar_cols.remove('Year_Period')
colors = sns.color_palette("Oranges_r").as_hex()

for column, color in zip(bar_cols, colors):       
        fig.add_trace(
            go.Bar(x=plot_df['Year_Period'], y=plot_df[column], name=column, marker_color=color),
        )
        
fig.add_trace(
            go.Scatter(x=plot_df.Year_Period ,y=plot_df.Target_National, name='Target_National')
    )        
        
        
fig.show()


KeyError: 'Palnned_National'

In [33]:
import plotly.graph_objs as go
import seaborn as sns

fig = go.Figure()

plot_df = pd.pivot(dsi_group.loc[dsi_group.Summary_Portfolio == 'Aseptic'], 
                     values=['Days_DSI'],
                     index=['Year_Period'],
                     columns=['Level2Region']
                    )


plot_df = pd.DataFrame(plot_df.to_records())

plot_df.columns = [hdr.replace("('Days_DSI', '", "").replace("')", "") for hdr in plot_df.columns]

plot_df['National'] = plot_df['North'] +  plot_df['Scotland'] + plot_df['South']

plot_df = plot_df.fillna(0)

bar_cols = ['North', 'South', 'Scotland', 'National']
colors = sns.color_palette("Oranges_r").as_hex()

for column, color in zip(bar_cols, colors):       
        fig.add_trace(
            go.Bar(x=plot_df['Year_Period'], y=plot_df[column], name=column, marker_color=color),
        )
        
# fig = go.Bar(x=[this_df.Year_Period, this_df.Parent_Category], y=this_df.Value, name=r, marker_color=c)

# fig = px.bar(dsi_group, x=["Year_Period", "Level2Region"], y=['Days_DSI'], title="Wide-Form Input")
fig.show()

In [ ]:
import plotly.express as px

wide_df = px.data.medals_wide()

fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")
fig.show()

In [121]:
import seaborn as sns

plot_df = dsi_group.loc[dsi_group.Summary_Portfolio == 'Aseptic']

bar_cols = plot_df.Level2Region.unique()
colors = sns.color_palette("Oranges_r").as_hex()

for column, color in zip(bar_cols, colors):
    print(color)
    print(column)

#ad3803
North
#e05206
Scotland
#f67824
South


In [169]:
dsi_table = dsi_table.drop(['Days_DSI', 'Days_DSI_Target'], axis=1)
dsi_table

,Site_SAP,Level2Region,Summary_Portfolio,Material,Year_Period,BucketsInPeriod,ZUC_TotalDemand,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target
0,5501,South,Glass,201945,2020_12,5,1137.0,32.49,313.0,313.0
1,5501,South,Glass,201946,2020_12,5,7416.0,211.89,1456.0,1456.0
2,5501,South,Aseptic,214634,2020_12,5,1899.0,54.26,958.0,958.0
3,5501,South,LPET,215668,2020_12,5,17829.0,509.40,2944.0,2944.0
4,5501,South,Monster,235715,2020_12,5,1420.0,40.57,579.0,579.0
...,...,...,...,...,...,...,...,...,...,...
7606,5558,North,Small Cans,252257,2021_04,4,68.0,2.43,10.0,10.0
7607,5558,North,Small Cans,264832,2021_04,4,440.0,15.71,678.0,678.0
7608,5558,North,Small Cans,265338,2021_04,4,36.0,1.29,541.0,541.0
7609,5558,North,Cans - Shrink,411526,2021_04,4,104420.0,3729.29,2469.0,2469.0


In [170]:
dsi_table['ZUC_PeriodDailyDemand_Next'] = dsi_table['ZUC_PeriodDailyDemand'].shift(1)
dsi_table

,Site_SAP,Level2Region,Summary_Portfolio,Material,Year_Period,BucketsInPeriod,ZUC_TotalDemand,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,ZUC_PeriodDailyDemand_Next
0,5501,South,Glass,201945,2020_12,5,1137.0,32.49,313.0,313.0,NaN
1,5501,South,Glass,201946,2020_12,5,7416.0,211.89,1456.0,1456.0,32.49
2,5501,South,Aseptic,214634,2020_12,5,1899.0,54.26,958.0,958.0,211.89
3,5501,South,LPET,215668,2020_12,5,17829.0,509.40,2944.0,2944.0,54.26
4,5501,South,Monster,235715,2020_12,5,1420.0,40.57,579.0,579.0,509.40
...,...,...,...,...,...,...,...,...,...,...,...
7606,5558,North,Small Cans,252257,2021_04,4,68.0,2.43,10.0,10.0,20.00
7607,5558,North,Small Cans,264832,2021_04,4,440.0,15.71,678.0,678.0,2.43
7608,5558,North,Small Cans,265338,2021_04,4,36.0,1.29,541.0,541.0,15.71
7609,5558,North,Cans - Shrink,411526,2021_04,4,104420.0,3729.29,2469.0,2469.0,1.29


In [173]:
dsi_table

,Site_SAP,Level2Region,Summary_Portfolio,Material,Year_Period,BucketsInPeriod,ZUC_TotalDemand,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,ZUC_PeriodDailyDemand_Next
0,5501,South,Glass,201945,2020_12,5,1137.0,32.49,313.0,313.0,NaN
1,5501,South,Glass,201946,2020_12,5,7416.0,211.89,1456.0,1456.0,32.49
2,5501,South,Aseptic,214634,2020_12,5,1899.0,54.26,958.0,958.0,211.89
3,5501,South,LPET,215668,2020_12,5,17829.0,509.40,2944.0,2944.0,54.26
4,5501,South,Monster,235715,2020_12,5,1420.0,40.57,579.0,579.0,509.40
...,...,...,...,...,...,...,...,...,...,...,...
7606,5558,North,Small Cans,252257,2021_04,4,68.0,2.43,10.0,10.0,20.00
7607,5558,North,Small Cans,264832,2021_04,4,440.0,15.71,678.0,678.0,2.43
7608,5558,North,Small Cans,265338,2021_04,4,36.0,1.29,541.0,541.0,15.71
7609,5558,North,Cans - Shrink,411526,2021_04,4,104420.0,3729.29,2469.0,2469.0,1.29


In [180]:
dsi_group = dsi_table[['Summary_Portfolio', 'Level2Region', 'Year_Period', 
           'ZUC_PeriodDailyDemand', 'ZUC_ClosingStock', 
           'ZUC_ClosingStock_Target']].groupby(['Summary_Portfolio', 'Level2Region', 'Year_Period']).sum().reset_index()


dsi_group['Days_DSI'] = dsi_group['ZUC_ClosingStock'] / dsi_group['ZUC_PeriodDailyDemand'].shift(1)
dsi_group['Days_DSI_Target'] = dsi_group['ZUC_ClosingStock_Target'] / dsi_group['ZUC_PeriodDailyDemand'].shift(1)

dsi_group_national = dsi_table[['Summary_Portfolio', 'Year_Period', 
           'ZUC_PeriodDailyDemand', 'ZUC_ClosingStock', 
           'ZUC_ClosingStock_Target']].groupby(['Summary_Portfolio', 'Year_Period']).sum().reset_index()

dsi_group_national['Planned_National'] = dsi_group_national['ZUC_ClosingStock'] / dsi_group_national['ZUC_PeriodDailyDemand'].shift(1)
dsi_group_national['Planned_Target'] = dsi_group_national['ZUC_ClosingStock_Target'] / dsi_group_national['ZUC_PeriodDailyDemand'].shift(1)

In [175]:
dsi_group

,Summary_Portfolio,Level2Region,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Days_DSI,Days_DSI_Target
0,Aseptic,North,2020_12,2893.29,28894.0,21889.0,NaN,NaN
1,Aseptic,North,2021_01,1053.70,22151.0,21939.0,7.65599024,7.58271725
2,Aseptic,North,2021_02,1239.06,26035.0,26035.0,24.70817121,24.70817121
3,Aseptic,North,2021_03,1280.10,29857.0,29857.0,24.09649250,24.09649250
4,Aseptic,North,2021_04,1334.99,32051.0,30909.0,25.03788767,24.14576986
...,...,...,...,...,...,...,...,...
232,Water,South,2020_12,12450.64,142002.0,142002.0,113.94068749,113.94068749
233,Water,South,2021_01,8675.11,119528.0,119528.0,9.60014907,9.60014907
234,Water,South,2021_02,10280.76,150720.0,150439.0,17.37384310,17.34145158
235,Water,South,2021_03,11822.33,203448.0,195978.0,19.78919846,19.06259848


In [176]:
dsi_group_national

,Summary_Portfolio,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Days_DSI,Days_DSI_Target
0,Aseptic,2020_12,4977.89,54251.0,43672.0,NaN,NaN
1,Aseptic,2021_01,2010.88,47194.0,46209.0,9.48072376,9.28284876
2,Aseptic,2021_02,2347.56,56290.0,56170.0,27.99271961,27.93304424
3,Aseptic,2021_03,2478.93,65003.0,64951.0,27.68960112,27.66745046
4,Aseptic,2021_04,2688.98,81769.0,68820.0,32.98560266,27.76197795
...,...,...,...,...,...,...,...
76,Water,2020_12,30173.69,333891.0,333888.0,14.68641094,14.68627899
77,Water,2021_01,21157.43,279646.0,279646.0,9.26787542,9.26787542
78,Water,2021_02,24919.02,354156.0,351490.0,16.73908409,16.61307635
79,Water,2021_03,28963.99,471934.0,463783.0,18.93870626,18.61160672


In [182]:
summary_portfolio = 'Aseptic'
    
plot_df = pd.pivot(dsi_group.loc[dsi_group.Summary_Portfolio == summary_portfolio], 
                     values=['Days_DSI', 'Days_DSI_Target'],
                     index=[ 'Summary_Portfolio', 'Year_Period'],
                     columns=['Level2Region']
                    )
    
plot_df = pd.DataFrame(plot_df.to_records())
plot_df.columns = [hdr.replace("('Days_DSI_Target', '", "Target_").replace("('Days_DSI', '", "Planned_").replace("')", "") for hdr in plot_df.columns]

plot_df

,Summary_Portfolio,Year_Period,Planned_North,Planned_Scotland,Planned_South,Target_North,Target_Scotland,Target_South
0,Aseptic,2020_12,NaN,2.24046622,209.02803738,NaN,2.24046622,175.62616822
1,Aseptic,2021_01,7.65599024,43.41508776,10.66162306,7.58271725,43.41508776,10.27484652
2,Aseptic,2021_02,24.70817121,77.31092437,28.38465066,24.70817121,77.31092437,28.25043898
3,Aseptic,2021_03,24.09649250,71.93730408,28.58712029,24.09649250,71.93730408,28.53657351
4,Aseptic,2021_04,25.03788767,72.07491820,39.02900378,24.14576986,72.07491820,28.39398307


In [181]:
dsi_group_national.loc[dsi_group_national.Summary_Portfolio == summary_portfolio]

,Summary_Portfolio,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Planned_National,Planned_Target
0,Aseptic,2020_12,4977.89,54251.0,43672.0,NaN,NaN
1,Aseptic,2021_01,2010.88,47194.0,46209.0,9.48072376,9.28284876
2,Aseptic,2021_02,2347.56,56290.0,56170.0,27.99271961,27.93304424
3,Aseptic,2021_03,2478.93,65003.0,64951.0,27.68960112,27.66745046
4,Aseptic,2021_04,2688.98,81769.0,68820.0,32.98560266,27.76197795


In [188]:
dsi_regional = pd.pivot(dsi_group, 
         values=['Days_DSI', 'Days_DSI_Target'],
         index=[ 'Summary_Portfolio', 'Year_Period'],
         columns=['Level2Region']
        )

dsi_regional = pd.DataFrame(dsi_regional.to_records())
dsi_regional.columns = [hdr.replace("('Days_DSI_Target', '", "Target_").replace("('Days_DSI', '", "Planned_").replace("')", "") for hdr in dsi_regional.columns]

dsi_regional

,Summary_Portfolio,Year_Period,Planned_North,Planned_Scotland,Planned_South,Target_North,Target_Scotland,Target_South
0,Aseptic,2020_12,NaN,2.24046622,209.02803738,NaN,2.24046622,175.62616822
1,Aseptic,2021_01,7.65599024,43.41508776,10.66162306,7.58271725,43.41508776,10.27484652
2,Aseptic,2021_02,24.70817121,77.31092437,28.38465066,24.70817121,77.31092437,28.25043898
3,Aseptic,2021_03,24.09649250,71.93730408,28.58712029,24.09649250,71.93730408,28.53657351
4,Aseptic,2021_04,25.03788767,72.07491820,39.02900378,24.14576986,72.07491820,28.39398307
...,...,...,...,...,...,...,...,...
76,Water,2020_12,13.30571720,0.93548180,113.94068749,13.30548867,0.93548180,113.94068749
77,Water,2021_01,8.77564516,12.76843929,9.60014907,8.77564516,12.76843929,9.60014907
78,Water,2021_02,15.72741082,24.73685542,17.37384310,15.64410514,22.95177370,17.34145158
79,Water,2021_03,17.79301103,26.49325313,19.78919846,17.78725117,25.84078470,19.06259848


,Summary_Portfolio,Year_Period,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target,Planned_National,Planned_Target
0,Aseptic,2020_12,4977.89,54251.0,43672.0,NaN,NaN
1,Aseptic,2021_01,2010.88,47194.0,46209.0,9.48072376,9.28284876
2,Aseptic,2021_02,2347.56,56290.0,56170.0,27.99271961,27.93304424
3,Aseptic,2021_03,2478.93,65003.0,64951.0,27.68960112,27.66745046
4,Aseptic,2021_04,2688.98,81769.0,68820.0,32.98560266,27.76197795
...,...,...,...,...,...,...,...
76,Water,2020_12,30173.69,333891.0,333888.0,14.68641094,14.68627899
77,Water,2021_01,21157.43,279646.0,279646.0,9.26787542,9.26787542
78,Water,2021_02,24919.02,354156.0,351490.0,16.73908409,16.61307635
79,Water,2021_03,28963.99,471934.0,463783.0,18.93870626,18.61160672


In [192]:
dsi_table.ZUC_ClosingStock = dsi_table.ZUC_ClosingStock.astype(float)
dsi_table.ZUC_ClosingStock_Target = dsi_table.ZUC_ClosingStock_Target.astype(float)
dsi_group = dsi_table[['Summary_Portfolio', 'Level2Region', 'Year_Period', 
           'ZUC_PeriodDailyDemand', 'ZUC_ClosingStock', 
           'ZUC_ClosingStock_Target']].groupby(['Summary_Portfolio', 'Level2Region', 'Year_Period']).sum().reset_index()


dsi_group['Days_DSI'] = dsi_group['ZUC_ClosingStock'] / dsi_group['ZUC_PeriodDailyDemand'].shift(1)
dsi_group['Days_DSI_Target'] = dsi_group['ZUC_ClosingStock_Target'] / dsi_group['ZUC_PeriodDailyDemand'].shift(1)

dsi_regional = pd.pivot(dsi_group, 
         values=['Days_DSI', 'Days_DSI_Target'],
         index=[ 'Summary_Portfolio', 'Year_Period'],
         columns=['Level2Region']
        )

dsi_regional = pd.DataFrame(dsi_regional.to_records())
dsi_regional.columns = [hdr.replace("('Days_DSI_Target', '", "Target_").replace("('Days_DSI', '", "Planned_").replace("')", "") for hdr in dsi_regional.columns]

#' DSI National
dsi_national = dsi_table[['Summary_Portfolio', 'Year_Period', 
           'ZUC_PeriodDailyDemand', 'ZUC_ClosingStock', 
           'ZUC_ClosingStock_Target']].groupby(['Summary_Portfolio', 'Year_Period']).sum().reset_index()

dsi_national['Planned_National'] = dsi_national['ZUC_ClosingStock'] / dsi_national['ZUC_PeriodDailyDemand'].shift(1)
dsi_national['Target_National'] = dsi_national['ZUC_ClosingStock_Target'] / dsi_national['ZUC_PeriodDailyDemand'].shift(1)

dsi_national = dsi_national[['Summary_Portfolio', 'Year_Period', 'Planned_National', 'Target_National']]

dsi_plot = pd.merge(dsi_regional, dsi_national, on=['Summary_Portfolio', 'Year_Period'], how='inner')

dsi_plot.loc[dsi_plot.Summary_Portfolio == summary_portfolio]

,Summary_Portfolio,Year_Period,Planned_North,Planned_Scotland,Planned_South,Target_North,Target_Scotland,Target_South,Planned_National,Target_National
0,Aseptic,2020_12,NaN,2.24046622,209.02803738,NaN,2.24046622,175.62616822,NaN,NaN
1,Aseptic,2021_01,7.65599024,43.41508776,10.66162306,7.58271725,43.41508776,10.27484652,9.48072376,9.28284876
2,Aseptic,2021_02,24.70817121,77.31092437,28.38465066,24.70817121,77.31092437,28.25043898,27.99271961,27.93304424
3,Aseptic,2021_03,24.09649250,71.93730408,28.58712029,24.09649250,71.93730408,28.53657351,27.68960112,27.66745046
4,Aseptic,2021_04,25.03788767,72.07491820,39.02900378,24.14576986,72.07491820,28.39398307,32.98560266,27.76197795


In [193]:
'Cans - asdasd'.startswith('Cans')

True

In [197]:
PROJECTED_DEMAND_TABLE = 'model.MTP_WF003_02_ProjectedDemand_REFRESH'
INVENTORY_TABLE = 'dbo.temp_iP_Summarised_materiallevel_REFRESH'
uom = 'ZUC'

In [205]:
conn = pyodbc.connect('Driver={SQL Server};'
                            f'Server={SERVER_NAME};'
                            f'Database={DATABASE_NAME};'
                            'Trusted_Connection=yes;')
      

sql = f'''
        WITH DSI_TABLE AS (
            SELECT	  
                CASE PD.[Site_SAP]
                        WHEN 'P67' THEN '5567'
                        WHEN 'P69' THEN '5569'
                        WHEN 'P14' THEN '5504'
                        ELSE PD.[Site_SAP]
                    END AS Site_SAP
                ,CASE PD.[Site_SAP]
                        WHEN 'P67' THEN 'North'
                        WHEN 'P69' THEN 'North'
                        WHEN 'P14' THEN 'South'
                        ELSE PD.[Level2Region]
                    END AS Level2Region
                ,PD.[Summary_Portfolio] 
                ,PD.[Material]
                ,CONCAT([Year],'_',format (CAST(PD.[CCEP_Period] AS INT), '0#')) AS Year_Period
                ,CASE CAST(PD.[CCEP_Period] AS INT) % 3 WHEN 0 THEN 5 ELSE 4 END AS BucketsInPeriod
                ,ROUND(convert(float,PD.[Value]),0) AS TotalDemand
                ,ROUND(ROUND(convert(float,PD.[Value]),0) / ((CASE CAST(PD.[CCEP_Period] AS INT) % 3 WHEN 0 THEN 5 ELSE 4 END)*7),2) AS PeriodDailyDemand
                ,ROUND(convert(float, IP.[Value]),0) AS ClosingStock
                ,ROUND(convert(float,IP_target.[Value]),0) AS ClosingStock_Target
            FROM [PACSQL_MTP].{PROJECTED_DEMAND_TABLE} PD
            INNER JOIN [PACSQL_MTP].{INVENTORY_TABLE} IP
                ON IP.Year_Period = CONCAT(PD.[Year],'_',format (CAST(PD.[CCEP_Period] AS INT), '0#'))
                AND IP.Material = PD.[Material]
                AND IP.Site_SAP = PD.Site_SAP
            INNER JOIN [PACSQL_MTP].{INVENTORY_TABLE} IP_target
                ON IP_target.Year_Period = CONCAT(PD.[Year],'_',format (CAST(PD.[CCEP_Period] AS INT), '0#'))
                AND IP_target.Material = PD.[Material]
                AND IP_target.Site_SAP = PD.Site_SAP
            WHERE 1=1
                AND PD.[Table] = 'Demand_Site_Material'
                AND PD.[KeyField] = 'ZUC_TotalDemand'
                AND IP.[KeyField] = 'ZUC_ClosingStock'
                AND IP_target.[KeyField] = 'ZUC_ClosingStockTarget'
        ),
        DSI_GROUPED AS (
            select Summary_Portfolio, Year_Period 
                    ,SUM(TotalDemand) AS Total_Demand 
                    ,SUM(PeriodDailyDemand) AS PeriodDailyDemand 
                    ,SUM(ClosingStock) AS ClosingStock
                    ,SUM(ClosingStock_Target) AS ClosingStock_Target
                    ,ROUND(SUM(ClosingStock) / SUM(PeriodDailyDemand), 2) AS DSI_Planned
                    ,ROUND(SUM(ClosingStock_Target) / SUM(PeriodDailyDemand), 2) AS DSI_Target			
            FROM DSI_TABLE
            GROUP BY Summary_Portfolio, Year_Period
        )
        SELECT Summary_Portfolio
        ,Year_Period
        ,Total_Demand
        ,PeriodDailyDemand
        ,ClosingStock
        ,ClosingStock_Target
        ,DSI_Planned
        ,DSI_Target
        ,ROUND((DSI_Target - DSI_Planned) *100 / DSI_Target, 2) AS DSI_Under_Target_Percent
        FROM DSI_GROUPED    
'''

data = pd.read_sql(sql,conn) 

planned_dsi = pd.pivot(data, values=['DSI_Planned'], index=['Summary_Portfolio'], columns=['Year_Period'])

In [206]:
flat = pd.DataFrame(planned_dsi.to_records())

flat.columns = [hdr.replace("('DSI_Planned', '", "").replace("')", "") for hdr in flat.columns]
flat

,Summary_Portfolio,2020_12,2021_01,2021_02,2021_03,2021_04,2021_05,2021_06,2021_07,2021_08,2021_09,2021_10,2021_11,2021_12,2022_01,2022_02,2022_03,2022_04
0,Aseptic,34.53,30.96,44.88,40.15,41.17,37.50,41.12,34.03,29.19,29.59,34.22,43.02,23.81,40.93,40.01,36.87,34.76
1,Cans - Board,10.94,17.36,21.37,17.08,19.28,19.67,19.35,18.89,20.14,18.84,20.32,21.11,12.71,22.97,23.35,19.48,18.58
2,Cans - Shrink,16.34,28.85,21.62,15.70,19.19,17.26,20.22,23.15,28.14,21.84,20.54,19.65,14.31,34.17,22.46,17.90,17.07
3,Cans - Singles,6.95,19.18,21.38,13.38,17.18,17.63,16.80,14.52,12.45,14.33,15.05,17.15,10.24,24.21,19.86,15.79,15.06
4,Capri Sun,23.24,24.81,30.16,25.25,27.05,34.19,33.75,28.05,24.15,27.05,26.41,29.94,21.54,32.07,32.56,29.49,27.94
5,GB,9.29,16.38,21.53,18.72,18.55,19.04,19.05,16.95,16.85,21.40,23.17,21.29,11.51,21.51,22.34,18.99,18.06
6,Glass,19.76,18.85,34.14,36.42,25.72,34.22,32.33,28.60,24.47,29.88,32.71,32.02,15.48,32.06,31.98,29.48,28.12
7,LPET,8.39,14.79,17.28,14.50,13.42,14.86,14.99,13.96,13.48,14.39,16.52,18.88,8.76,16.09,17.20,15.31,14.56
8,Monster,36.15,39.05,46.05,53.70,49.01,42.52,46.16,42.14,45.24,45.18,40.88,38.44,37.14,57.75,51.29,44.28,41.42
9,OTHER,16.53,28.30,38.37,40.66,34.70,34.63,33.99,40.40,31.50,35.11,39.30,35.46,23.33,40.94,40.00,36.88,34.77
